In [2]:
import pandas as pd  
playlists_df = pd.read_csv("temp/playlists_df.csv", keep_default_na=False)

,Folder,List,Smart,Length,Position,Artist,Name,Duration,Codec,Creation,Date Modified,Location,pc_loc,mac_loc,pc_mac_loc,linkable,linked,mac_pc_loc,level,color,title_song_color
0,LOCAL,Mala qualitat,True,275,0,Cage The Elephant,Back Against The Wall,3:48,MP3,2022-11-09 12:58:13,,D:\MUSIQUE\PASAR A MAC\Mala qualitat\Cage The ...,D:\MUSIQUE\PASAR A MAC\Mala qualitat\Cage The ...,file:///Users/legoteque/Music/Itunes%20Media/C...,/Volumes/PASAR A MAC/Mala qualitat/Cage The El...,True,False,\\169.254.170.1\Musica\Itunes Media\Cage The E...,-1,darkgreen,red
1,LOCAL,Mala qualitat,True,275,1,The Kooks,Bad Habit,3:41,MP3,2022-11-09 12:58:27,,D:\MUSIQUE\PASAR A MAC\Mala qualitat\The Kooks...,D:\MUSIQUE\PASAR A MAC\Mala qualitat\The Kooks...,file:///Users/legoteque/Music/Itunes%20Media/T...,/Volumes/PASAR A MAC/Mala qualitat/The Kooks -...,True,True,\\169.254.170.1\Musica\Itunes Media\The Kooks ...,5,red,red
2,LOCAL,Mala qualitat,True,275,2,Git Fresh,Booty Music,3:38,MP3,2022-11-09 12:58:33,,D:\MUSIQUE\PASAR A MAC\Mala qualitat\Git Fresh...,D:\MUSIQUE\PASAR A MAC\Mala qualitat\Git Fresh...,file:///Users/legoteque/Music/Itunes%20Media/G...,/Volumes/PASAR A MAC/Mala qualitat/Git Fresh -...,True,True,\\169.254.170.1\Musica\Itunes Media\Git Fresh ...,4,orange red,red
3,LOCAL,Mala qualitat,True,275,3,Sidonie,El Bosque,4:29,MP3,2022-11-09 12:58:33,,D:\MUSIQUE\PASAR A MAC\Mala qualitat\Sidonie -...,D:\MUSIQUE\PASAR A MAC\Mala qualitat\Sidonie -...,file:///Users/legoteque/Music/Itunes%20Media/S...,/Volumes/PASAR A MAC/Mala qualitat/Sidonie - E...,True,True,\\169.254.170.1\Musica\Itunes Media\Sidonie - ...,5,red,red
4,LOCAL,Mala qualitat,True,275,4,We Are Standard,Can I Count on You? (Monkey Rave Remix),4:40,MP3,2022-11-09 12:58:45,,D:\MUSIQUE\PASAR A MAC\Mala qualitat\We Are St...,D:\MUSIQUE\PASAR A MAC\Mala qualitat\We Are St...,file:///Users/legoteque/Music/Itunes%20Media/W...,/Volumes/PASAR A MAC/Mala qualitat/We Are Stan...,True,True,\\169.254.170.1\Musica\Itunes Media\We Are Sta...,4,orange red,red
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55090,root,Random passar a mac,False,866,861,Kevin Gates,Big Hittah,3:02,MP3,2022-11-03T11:49:28Z,2022-11-03T11:48:40Z,\\169.254.170.1\Musica\Itunes Media\Kevin Gate...,,file:///Users/legoteque/Music/Itunes%20Media/K...,,False,False,\\169.254.170.1\Musica\Itunes Media\Kevin Gate...,-1,black,
55091,root,Random passar a mac,False,866,862,Charli XCX;Kim Petras;Jay Park,Unlock it (Lock It),3:53,MP3,2021-10-06T22:31:11Z,2022-11-03T11:34:25Z,\\169.254.170.1\Musica\Itunes Media\Charli XCX...,,file:///Users/legoteque/Music/Itunes%20Media/C...,,False,False,\\169.254.170.1\Musica\Itunes Media\Charli XCX...,-1,black,
55092,root,Random passar a mac,False,866,863,Bruno Mars,Talking to the Moon,3:38,MP3,2021-09-24T09:36:29Z,2022-11-03T11:36:40Z,\\169.254.170.1\Musica\Itunes Media\Bruno Mars...,,file:///Users/legoteque/Music/Itunes%20Media/B...,,False,False,\\169.254.170.1\Musica\Itunes Media\Bruno Mars...,-1,black,
55093,root,Random passar a mac,False,866,864,Sia,Together (Initial Talk Remix),3:18,MP3,2022-11-03T11:49:28Z,2022-11-03T11:48:46Z,\\169.254.170.1\Musica\Itunes Media\Sia - Toge...,,file:///Users/legoteque/Music/Itunes%20Media/S...,,False,False,\\169.254.170.1\Musica\Itunes Media\Sia - Toge...,-1,black,


quan fem click amb el ratoli sobre un tema, fem un widget.selection (señalem l'item del tree) + un widget.focus. La combinació del selection, quan hi ha el focus sobre la llista es el que llança la funció asociada al tag_bind

quan programem, podem fer aquests dos passos per separat. el widget.selection solament ens marcará la casella seleccionada si la llista no te el focus i si hi posem el widget.focus sibre seu, el selection llançará l'event asociat al widget

In [1]:
timings = False
load_export_path = False

import time
import os, re, random, math, vlc, json, mutagen
import pandas as pd
import tkinter as tk
import tkinter.simpledialog
from tkinter import ttk, filedialog, messagebox
from legoteca import ItunesLibrary, AudioFile, Spotify
from legoteca import ms_to_string, remote_xml_conected_assert
from utilities import Linker
from utilities import extract_pic_from_metadata, bpms_iguals_o_multiples, key_from_value
from utilities import format_spaces

#els del mateix arxiu. No seran necessaris
from tags import TagsReader, TagsEditor, MusicPlayer
from tags import PATTERN, TYPE_SUBGENRE, DESC_SUBGENRE, HEART, STAR, RATING, GENERATION

import warnings
warnings.filterwarnings("ignore", 'This pattern has match groups')


#CONSTANTS

with open('data.json') as json_file:
    data = json.load(json_file)  
EXPORT_PATH = data["DATA"]["EXPORT_PATH"]

PATTERN = {"genre": r"^\!\d\d \![\w\&]+\b",
           "subgenre": r"\B![A-Z][\w\&]+\b",
           "desc_subgenre": r"\B\.\w{2,5}\b",
           "rating": r"\B\*\B|\B\'z{1,4}p?\'\B",
           "club": r"\B\@\S{3,5}\b",
           "generation": r"\B\:[C,B,X,Y,Z]\b",
           "year": r"^\d{4}$",
           "bpm": r"^(([4-9][0-9])|1[0-9][0-9]|2[0-9][0-9])$",
           "hashtag": r"\B\#\S+"}

#valors i noms de les variables de checkbox (no és una descripció, serveix per definirles)
TYPE_SUBGENRE = {"rebeat": ".rb", "doubletempo": ".dt", "halftempo": ".ht", "dembowed": ".dmbw", 
                 "cover": ".co", "remix": ".rmx", "mashup": ".mu",
                 "alternative": ".AH",  "mainstream": ".MS"}

DESC_SUBGENRE = {".mel": "Melodic", ".hpy": "Happy", ".drk": "Dark",
                 ".chil": "Chill", ".sax": "Saxo",
                 ".epc": "Epic", ".hyp": "Hypnotic", ".atm": "Atmospheric", ".etn": "Etnic",
                 ".teen": "Teenager", ".auTu": "Autotune",
                 ".orch": "Orchestra", ".ins": "Instrumental", ".voc": "Vocal"}

HEART, STAR = "\U0001F9E1", "\U00002B50"
RATING = {"*": "*", STAR:"'z'", STAR*2: "'zz'", STAR*3: "'zzz'", STAR*4: "'zzzz'"}
RATING_IT = {"": "", "20": STAR, "40": STAR*2, "60": STAR*3, "80": STAR*4, "100": STAR*5}

GENERATION = {":Z": "Z Gen", ":Y": "Y Gen", ":X": "X Gen", ":B": "Boomer", ":C": "Classic"}


def yesno_to_bool(answer):
        if answer == "yes": return True
        if answer == "no": return False 

if timings: time0 = time.time()
    

    
    
    
    



class Interface(tk.Tk):
    def __init__(self, it=None):
        super().__init__()
        self.title("Playlists Tagger")
        #self.geometry("1200x1000")
        self.attributes('-fullscreen', True)
        
        #afegim label loading
        self.loading_lbl = tk.Label(self, text="Loading...", 
                                    bg="white", fg="red", font=("Courier", 30), relief="solid", borderwidth=5)
        self.loading_lbl.place(relx=.5, rely=.5, anchor=tk.CENTER)

        #mostrem el construït
        self.update()
        
        #importem els ultims unics desats
        self.load_last_saved_data()
        
        # make the top right close button minimize: self.iconify)
        #self.protocol("WM_DELETE_WINDOW", self.iconify)
        #previene la interaccion con la ventana
        #self.attributes('-disabled', True)
        # make Esc exit the program
        #self.bind('<Escape>', lambda e: self.close_app())
        
        
        #afegim shortcuts
        self.bind('<Control-d>', lambda e: self.mybpm_mult("double"))
        self.bind('<Control-h>', lambda e: self.mybpm_mult("half"))
        self.bind('<Control-r>', lambda e: self.mybpm_mult("rebeat"))

        # create a menu bar with Minimize an Exit command
        menubar = tk.Menu(self)
        self.config(menu=menubar)
        
        menubar.add_command(label="Exit", command=self.close_app)
        menubar.add_command(label="Minimize", command=self.minimize_win)
        
        filemenu = tk.Menu(menubar, tearoff=0)
        filemenu.add_command(label="Add a path to local lists", command=self.add_local_list)
        filemenu.add_separator()
        filemenu.add_command(label="Create audio playlist to import in Itunes", command=self.create_playlist_to_export)
        
        listmenu = tk.Menu(menubar, tearoff=0)
        listmenu.add_command(label="Prova pop-up")
        
        menubar.add_cascade(label="Utilities", menu=filemenu)
        menubar.add_cascade(label="List", menu=listmenu)
        
        #patch per poder posar colors als treeviews
        s = ttk.Style()
        def fixed_map(option):
            return [elm for elm in s.map('Treeview', query_opt=option) if elm[:2] != ('!disabled', '!selected')]
        s.map('Treeview', foreground=fixed_map('foreground'), background=fixed_map('background'))
        
        #construim layout
        self.build_media_frame()
        
        self.title_song = tk.StringVar(value="")
        self.title_song_lbl = tk.Label(self, textvariable=self.title_song, font=("Courier", 15), 
                                  padx=2, pady=2, fg="blue")
        self.title_song_lbl.pack()
        
        lower_frame = tk.Frame(self)
        lower_frame.columnconfigure(1, weight=1)
        lower_frame.pack(fill=tk.BOTH, padx=10)
        
        label = tk.Label(lower_frame, text="", bg="#444")
        label.grid(row=0, column=1, rowspan=2, sticky="NSEW")            
        
        #construim frame de tags
        tags_frame = tk.Frame(lower_frame, relief=tk.SOLID, borderwidth=1)
        tags_frame.grid(column=0, row=0, ipadx=10, ipady=10, sticky="NSEW")
        self.itunes_tags = TagsReader(self, tags_frame, frame="Itunes")
        
        #carreguem primer l'editor, doncs el reader necessita trobarlo per poder dirigirli el load
        self.editor = TagsEditor(self, lower_frame)
        # location_width defineix el tamany del frame tagsReader i ho resta de TagsEditor
        self.reader = TagsReader(self, tags_frame, frame="Metadata", location_width=85)
        
        self.build_spotify_frame()
        
        #carreguem folders
        self.build_tree_folders()
        
        #inicialitzem i instanciem variables de reproduccio
        self.ct = CurrentTrack(self, loaded=False)
        self.lm = ListManager(self)
        self.enable_trace_vars = True #habilitem binds (de moment a list_selected)
        self.media = None

#SHORTCUTS
    def mybpm_mult(self, multiple):
        if self.editor.editor_status.get() == "EDITOR": return
        mybpm = self.editor.tracknumber.get()        
        if multiple == "double":
            double = str(int(mybpm) * 2)
            self.editor.tracknumber.set(double)
        elif multiple == "half":
            half = str(math.ceil(int(mybpm) / 2)) #math.ceil arrodoneix a int cap a dalt
            self.editor.tracknumber.set(half)
        elif multiple == "rebeat":
            rebeat = str(math.ceil(int(mybpm) * 1.3125))
            self.editor.tracknumber.set(rebeat)

#----------------------ITUNES LIBRARY IMPORT & LISTS OF UNIQUES----------------------------
    #funció a la que passant un unique ens retorna els uniques sota un pattern
    def uniques(self, column, pattern, genre=None):
        if genre == None:
            data = self.tracks_df[column].unique()
        else:
            data = self.tracks_df[self.tracks_df.Genre == genre]
            data = data[column].unique()
        matches = []
        for value in data:
            found = re.findall(pattern, value)
            matches = matches + found
            matches = set(matches)
            matches = list(matches)
            matches.sort()
        return matches
    
    def build_uniques(self):
        #gèneres únics (son afegibles per l'usuari temporalment)
        genres_l = self.uniques("Genre", PATTERN["genre"])
        
        #subgèneres únics (son afegibles per l'usuari temporalment)
        self.subgenres_l = self.uniques("Album", PATTERN["subgenre"])
        
        #subgèneres per gènere
        self.sub_x_genre_dic = {genre: self.uniques("Album", PATTERN["subgenre"], genre) for genre in genres_l}
        
        #subgèneres descriptius únics (definits)
        self.desc_subgen_dic = DESC_SUBGENRE.copy()
        #subgéneres descriptius unics no definits(els busca a la biblioteca)
        desc_subgen_l = self.uniques("Album", PATTERN["desc_subgenre"])
        #muntem diccionari de subgèneres únics agafant la descripció de desc_subgen_dic, si la troba. Sinó igual.
        desc_subgens, desc_subtypes = list(DESC_SUBGENRE.keys()), list(TYPE_SUBGENRE.values())
        new_desc_subgen_l = [desc for desc in desc_subgen_l if desc not in desc_subgens + desc_subtypes]
        for desc in new_desc_subgen_l: self.desc_subgen_dic[desc] = desc
        
        #clubs únics
        self.clubs_l = self.uniques("Composer", PATTERN["club"])
        
        #hashtags unics
        hashtags_l = self.uniques("Comments", PATTERN["hashtag"])
        self.hashtags_l = [h.replace("#", "") for h in hashtags_l]
        
        #creem un diccionari amb els unics cada cop que llegim per la propera vegada que obrim poder omplir els comboxes
        uniques_dic = {"subgenres_l": self.subgenres_l,
                       "sub_x_genre_dic": self.sub_x_genre_dic,
                       "desc_subgen_dic": self.desc_subgen_dic,
                       "clubs_l": self.clubs_l,
                       "hashtags_l": self.hashtags_l}
        
        with open('temp/uniques.json', 'w') as f:
            json.dump(uniques_dic, f)

    def load_last_saved_data(self):
        with open('temp/uniques.json') as f:
            uniques_dic = json.load(f)
            
        self.subgenres_l = uniques_dic["subgenres_l"]
        self.sub_x_genre_dic = uniques_dic["sub_x_genre_dic"]
        self.desc_subgen_dic = uniques_dic["desc_subgen_dic"]
        self.clubs_l = uniques_dic["clubs_l"]
        self.hashtags_l = uniques_dic["hashtags_l"]
        
        #importem l'última versio carregada de playlists_df
        self.playlists_df = pd.read_csv("temp/playlists_df.csv", keep_default_na=False)
        self.playlists_df = self.playlists_df.astype({"Smart": bool, "linkable": bool, "linked": bool})
        #importem l'ultima versió llegida de tracks_df
        self.tracks_df = pd.read_csv("temp/tracks_df.csv", dtype={"BPM": "str"}, keep_default_na=False)
        #importem l'ultima versió llegida de finder_df
        finder_df = pd.read_csv("temp/finder_df.csv")
        finder_df.fillna("", inplace=True)
        self.linker = Linker(self.tracks_df, finder_df=finder_df)
    
    def import_itunes_library_and_linker(self):
        it = ItunesLibrary(xml_local_path=None, max_tracks_pl=2000)
        #If there are duplicated tracks
        if it.hay_duplicated_locs | it.hay_duplicated_tracks:
            self.loading_lbl.destroy()
            duplicated = pd.DataFrame()
            if it.hay_duplicated_locs: duplicated = it.duplicated_locs.copy()
            if it.hay_duplicated_tracks: duplicated = pd.concat((duplicated, it.duplicated_tracks))

            duplicated.to_csv("duplicated.csv")

            msg = "\n\nCreat 'duplicated.csv' per consultar els duplicats"
            msg += "\n\n Soluciona aquests duplicats i recarrega la biblioteca"
            msg = it.msg + msg
            messagebox.showwarning(title="Info", message=msg)
            return        
        
        self.tracks_df = it.tracks_df
        self.linker = Linker(self.tracks_df)
        
        #apliquem filtrat de playlists d'itunes
        #pattern = r"^([1-9](A|B))|(1[0-2](A|B))"
        pattern = r"^(1[0-2]|[1-9])(A|B)"
        #cond1 = it.playlists_df.List.str.contains(pattern)
        cond1 = it.playlists_df.List.str.match(pattern)
        cond2 = it.playlists_df.Folder.str.contains("^--")
        it_playlists_df = it.playlists_df[~cond1 & ~cond2].copy()
        
        local_playlists_df = self.playlists_df[self.playlists_df.Folder == "LOCAL"]
        self.playlists_df = pd.concat([local_playlists_df, it_playlists_df])
        self.playlists_df.reset_index(drop=True, inplace=True)
        self.playlists_df.fillna("", inplace=True)
        
        return it.msg
    
        
#BUILD MEDIA
    def build_media_frame(self):   
        media_frame = tk.Frame(self, relief=tk.SOLID, borderwidth=2)
        media_frame.columnconfigure(1, weight=1)
        media_frame.pack(fill=tk.BOTH, padx=10)
        
        navigator_frame = tk.Frame(media_frame, relief='raised', borderwidth=1, bg="#333")
#         -----------
        itunes_bar = tk.Frame(navigator_frame, bg="black")
        
        import_library_btn = tk.Button(itunes_bar, text="Import Library", command=self.import_itunes_to_browser)
        search_library_btn = tk.Button(itunes_bar, text="Search in Library")
        
        import_library_btn.grid(row=0, column=0, padx=2)
        search_library_btn.grid(row=0, column=1, padx=2)
#         -----------
        lists_bar = tk.Frame(navigator_frame, bg="black")
    
        add_local_list_btn = tk.Button(lists_bar, text="Add", command=self.add_local_list)
        self.remove_list_btn = tk.Button(lists_bar, text="Remove", command=self.delete_local_list)
        self.reload_list_btn = tk.Button(lists_bar, text="Reload", command=self.reload_local_list)
        self.update_list_btn = tk.Button(lists_bar, text="Update", command=self.refresh_list_from_browser)
        
        add_local_list_btn.grid(row=0, column=0, padx=2)
#         -----------
        navigator_bar = tk.Frame(navigator_frame, bg="black")
                
        self.sortby_cb = tk.StringVar(value="Creation (desc.)")
        self.sort_cb = ttk.Combobox(navigator_bar, textvariable=self.sortby_cb, height=30, width=11,
                               values=["Artist", "Position", "Creation (asc.)", "Creation (desc.)", "Modified (asc.)", "Modified (desc.)"])
        self.sort_cb.bind('<<ComboboxSelected>>', self.list_selected)
        
        
        self.origin_route = tk.StringVar(value="Local mode")
        self.origin_route_cb = ttk.Combobox(navigator_bar, textvariable=self.origin_route, 
                               values=["Find remotes", "Local mode"], height=30, width=12)
                               
        
        self.list_btn1 = tk.Button(navigator_bar, text="Export playlist", command=self.create_playlist_to_export)
        
        self.list_btn2 = tk.Button(navigator_bar, text="Add hashtag", command=self.add_hashtag_to_list)
                                               
        #self.list_btn3 = tk.Button(navigator_bar, text="Build new tracks folder", command=self.build_new_tracks_folder)                                       
                                               
#         -----------
        self.folder_list = ttk.Treeview(navigator_frame, height=13, selectmode="browse")
        self.folder_list.tag_bind("selected", "<<TreeviewSelect>>", self.folder_selected)
        self.folder_list.heading("#0", text="Folders")
        
        self.list_list = ttk.Treeview(navigator_frame, height=13, selectmode="browse")
        self.list_list.tag_bind("selected", "<<TreeviewSelect>>", self.list_selected)
        
        self.list_list.heading("#0", text="Lists")
        
        self.audio_list = ttk.Treeview(navigator_frame, height=13, columns=("artist", "title", "level"), selectmode="browse")
        self.audio_list.tag_bind("selected", "<<TreeviewSelect>>", self.audio_selected)
        self.audio_list_sb = ttk.Scrollbar(navigator_frame, orient="vertical", command=self.audio_list.yview)
        self.audio_list.configure(yscrollcommand=self.audio_list_sb.set)
        self.audio_list.heading("#0", text="Nº")
        self.audio_list.column("#0", minwidth=0, width=41, stretch=tk.NO)
        self.audio_list.heading("artist", text="Artist")
        self.audio_list.column("artist", minwidth=0, width=250, stretch=tk.NO)
        self.audio_list.heading("title", text="Title")
        self.audio_list.column("title", minwidth=0, width=449, stretch=tk.NO)
        self.audio_list.heading("level", text="L")
        self.audio_list.column("level", minwidth=0, width=25, stretch=tk.NO)
        
        
        itunes_bar.grid(row=0, column=0, sticky="NSEW")
        lists_bar.grid(row=0, column=1, sticky="NSEW")
        navigator_bar.grid(row=0, column=2, sticky="NSEW")

        self.folder_list.grid(row=1, column=0)
        self.list_list.grid(row=1, column=1)
        self.audio_list.grid(row=1, column=2)
        self.audio_list_sb.grid(row=1, column=3, sticky="nse")
#---------------------------------------------
        self.route_song = tk.StringVar(value="")
        route_song_lbl = tk.Label(media_frame, textvariable=self.route_song, font=("Courier", 11), 
                                  fg="white", bg="#666", height=3, borderwidth=4, relief=tk.SOLID)
   
        self.player_frame = tk.Frame(media_frame, relief='raised', borderwidth=30, bg="#044")
    
        back_btn = tk.Button(self.player_frame, text="back", relief="raised", borderwidth=5,
                                  command = lambda: self.change_song(-1))
        jump15back = tk.Button(self.player_frame, text="-15", relief="raised", borderwidth=5, 
                                    command = lambda: self.jump(-15))
        play_btn = tk.Button(self.player_frame, text="play", relief="raised", borderwidth=5, command=self.play)
        pause_btn = tk.Button(self.player_frame, text="pause", relief="raised", borderwidth=5, command=self.pause)
        stop_btn = tk.Button(self.player_frame, text="stop", relief="raised", borderwidth=5, command=self.stop)
        jump15frwd = tk.Button(self.player_frame, text="+15", relief="raised", borderwidth=5, 
                                    command = lambda: self.media.jump(15))
        next_btn = tk.Button(self.player_frame, text="next", relief="raised", borderwidth=5,
                                  command = lambda: self.change_song(1))
        
        self.time_song = tk.StringVar(value="")
        self.time_song_lbl = tk.Label(self.player_frame, textvariable=self.time_song, font=("Courier", 20),
                                      bg="#044", fg="white", padx=2, pady=10, foreground="yellow")
        self.perc_song = tk.StringVar(value="")
        self.perc_song_lbl = tk.Label(self.player_frame, textvariable=self.perc_song, font=("Courier", 20),
                                      bg="#044", fg="white", padx=2, pady=10, foreground="yellow")
        self.song_dur = tk.StringVar(value="")
        self.song_dur_lbl = tk.Label(self.player_frame, textvariable=self.song_dur, font=("Courier", 15),
                                      bg="#044", fg="white", padx=2, pady=10, foreground="yellow")
        
        self.random = tk.BooleanVar(self)
        random_cb = ttk.Checkbutton(self.player_frame, text="Shuffle ", variable=self.random)
        
        self.volume_sc = tk.Scale(self.player_frame, from_=1, to=100, orient=tk.HORIZONTAL, showvalue=0, resolution=1, 
                       command=self.set_volume)
        self.volume_sc.set(100)

        back_btn.grid(row=0, column=0)
        jump15back.grid(row=0, column=1)
        play_btn.grid(row=0, column=2)
        pause_btn.grid(row=0, column=3)
        stop_btn.grid(row=0, column=4)
        jump15frwd.grid(row=0, column=5)
        next_btn.grid(row=0, column=6)
        self.time_song_lbl.grid(row=1, column=0, columnspan=3)
        self.perc_song_lbl.grid(row=1, column=3, columnspan=2)
        self.song_dur_lbl.grid(row=1, column=5, columnspan=2)
        random_cb.grid(row=2, column=5, columnspan=2)
        self.volume_sc.grid(row=2, column=0, columnspan=5, sticky="EW")
#---------------------------------------------
        self.label_art = tk.Label(media_frame, bg="white", relief="solid")
        #carreguem l'art
        image = extract_pic_from_metadata(None, size=(311,311), return_logo=True)
        self.label_art.config(image=image)
        self.label_art.image=image
        
        navigator_frame.grid(row=0, column=0)
        route_song_lbl.grid(row=0, column=1, sticky="NEW")
        self.player_frame.grid(row=0, column=1)
        self.label_art.grid(row=0, column=2, sticky="NSEW")

    def build_spotify_frame(self):
        frame5 = tk.Frame(self, bg="black")
        frame5.columnconfigure(0, weight=1)
        frame5.pack(padx=10, pady=5, fill=tk.BOTH, expand=True) 

#---------------------------------------PLAYER FUNTIONS----------------------------------

    def player_instance(self, playing_path, autoadvance):
        del self.media
        #generem instancia del nou track
        self.media = MusicPlayer(playing_path)
        pevent = self.media.audio.event_manager()
        pevent.event_attach(vlc.EventType.MediaPlayerPositionChanged, self.pos_callback)
        if autoadvance:
            pevent.event_attach(vlc.EventType.MediaPlayerEndReached, lambda e: self.change_song(1))
        
    def pos_callback(self, event):
        self.time_song.set(self.media.playing_time())
        position = int(self.media.audio.get_position() * 100)
        self.perc_song.set(str(position)+"%")
        
    def play_path(self, playing_path=None, prelisten=None):
        if prelisten != None:
            if prelisten == "select": playing_path = self.ct.itunes_link_loc[1]
            elif prelisten == "button":
                if self.ct.playing_origin == "Local": playing_path = self.ct.itunes_link_loc[1]
                else: playing_path = self.ct.pc_loc
            autoadvance = False
            if self.media.audio.is_playing():
                if self.media.song_path == playing_path: return
                else: self.media.audio.stop() #si seleccionem qualsevol altre track aturem amb release
            #check if file exists
            if not os.path.isfile(playing_path):
                msg = f"No connection to {playing_path}."
                messagebox.showinfo(title="Info", message=msg)   
                if hasattr(self.itunes_tags, 'window'): self.itunes_tags.window.destroy()   
                return          
            
            #canviem el color del player si es prelisten
            self.player_frame.config(bg="red")
            self.time_song_lbl.config(bg="red", foreground="black")
            self.perc_song_lbl.config(bg="red", foreground="black")
            self.song_dur_lbl.config(bg="red", foreground="black")
        else: 
            autoadvance = True
            #color del player
            self.player_frame.config(bg="#044")
            self.time_song_lbl.config(bg="#044", foreground="yellow")
            self.perc_song_lbl.config(bg="#044", foreground="yellow")
            self.song_dur_lbl.config(bg="#044", foreground="yellow")
        
        self.player_instance(playing_path, autoadvance)
        duration = mutagen.File(playing_path).info.length
        duration = ms_to_string(duration, metric="seconds")
        self.song_dur.set(duration)
        self.play()
        
    def play(self):
        if not self.ct.loaded: return
        self.time_song.set("Loading")
        self.perc_song.set("")
        self.media.audio.play()
        
    def stop(self):
        if not self.ct.loaded: 
            #print("no hi ha current track")
            return
        #print("hi ha current track")
        if hasattr(self.media, 'audio'):
            #print("hi ha self.media")
            if self.media.audio.is_playing(): 
                self.media.audio.stop()
                self.time_song.set("0:00")
                self.perc_song.set("0%")
        #else: print("no hi ha self.media")
        
    def pause(self):
        if not self.ct.loaded: return
        self.media.audio.pause()
            
    def jump(self, sec):
        if not self.ct.loaded: return
        if self.media.audio.is_playing(): self.media.jump(sec)
            
    def change_song(self, jump):
        if not self.ct.loaded: return
        if self.media.audio.is_playing(): self.media.audio.stop()
        folder_iid, lista_iid = self.ct.folder_iid, self.ct.list_iid
        audio_iid, list_len = self.ct.audio_iid, self.ct.Length
  
        #si som a una llista diferent (except controla la possiblitat que no hi hagi llista seleccionada) no fa res
        try:
            list_selection_iid = self.list_list.selection()[0]
        except: return
        if list_selection_iid != lista_iid: return
        #si esta ubicat en el seu folder i llista posem el focus sobre el track actual
        self.audio_list.focus(audio_iid)
        if self.random.get():
            possible_next = list(range(int(list_len)))
            possible_next.remove(int(audio_iid))
            next_audio_iid = str(random.choice(possible_next))
        else:
            next_audio_iid = str(int(audio_iid) + jump)
        #si el track cridat esta fora de rang evitem l'excepció
        try:
            self.audio_list.selection_set(next_audio_iid)
        except: return
            
    def set_volume(self, e):
        if not self.ct.loaded: return
        value = int(math.log(self.volume_sc.get(), 1.047))
        self.media.audio.audio_set_volume(value)


    def close_app(self):
        self.stop()
        self.destroy()
        
    def minimize_win(self):
        self.wm_state('iconic')

        
#-----------------------------------FUNCIONS PER EDITAR LLISTES-------------------------- 
    def save_playlists_df(self):
        #esborrem columnes iid si les hem creat amb les funcions de llista
        playlists_df = self.playlists_df.copy()
        del_columns = ["folder_iid", "list_iid", "audio_iid"]
        for col in del_columns:
            if col in playlists_df: playlists_df.drop(columns=col, inplace=True)
        
        playlists_df.to_csv("temp/playlists_df.csv", index=False)     
        
    def return_folder_list_selected(self, return_ids=False):
        list_iid = self.list_list.selection()[0]
        lista = self.list_list.item(list_iid, option="text")
        values = self.list_list.item(list_iid, option="values")
        folder_iid, folder = values[0], values[1]
        
        if return_ids: return folder, lista, folder_iid, list_iid
        return folder, lista
    
    def append_list_to_playlists(self, new_list_df):
        new_list_df = new_list_df.astype({"Smart": bool, "linkable": bool, "linked": bool})
        #ubiquem la nova llista darrera de les locals existents
        local_playlists_df = self.playlists_df[self.playlists_df.Folder == "LOCAL"].copy()
        local_playlists_df = pd.concat([local_playlists_df, new_list_df])
        #eliminem locals de playlists_df 
        index = self.playlists_df[self.playlists_df.Folder == "LOCAL"].index
        self.playlists_df.drop(index=index, inplace=True)
        #afegim playlists locals ordenades davant itunes playlists
        self.playlists_df = pd.concat([local_playlists_df, self.playlists_df])
        self.playlists_df.reset_index(drop=True, inplace=True)
        self.playlists_df.fillna("", inplace=True)
        
        self.save_playlists_df()
             
    def local_list_selection_after_edit(self, event):
        if event in ["add", "delete list", "refresh"]:
            #esborrem folders per si no estem a LOCAL, o no nhi ha, al donar-li add, 
            #o per si es un delete list i queda sense locals
            self.delete_audio_tree(lista=True, folder=True)
            self.build_tree_folders()
        
            #folder_iid=0 sera LOCAL si existeix, sino sera el primer que trobi
            self.folder_list.selection_set(0)
            self.folder_selected(None)

            local_playlists = self.playlists_df[self.playlists_df.Folder == "LOCAL"]
            bigger_list_iid = len(local_playlists["List"].unique()) - 1
            
            if not bigger_list_iid < 0: 
                #self.lm._folder_list_selected = [None, None] #forcem a que la llista seleccionada sigui carregada
                self.list_list.selection_set(bigger_list_iid)
                
        #if event == "refresh": self.ct.loaded = False #perque no subiqui sobre lultim audio_iid
        
        self.list_selected(event)
        
        if event == "delete track":
            self.audio_list.selection_set(self.ct.audio_iid)
            self.audio_selected("None")
            
#         elif event == "refresh":
#             self.audio_list.selection_set(0)
#             self.audio_selected("None")

#new_tracks_folder especifica a select_list la seleccio de export
    def add_local_list(self): 
        def extract_path(pathfile):
            pathfile = pathfile.split(os.sep)[:-1]
            return os.sep.join(pathfile)
        
        #busquem els paths locals ja agregats a les llistes
        path_s = self.playlists_df[self.playlists_df.Folder == "LOCAL"]["pc_loc"].apply(extract_path)

        path = filedialog.askdirectory(initialdir=EXPORT_PATH, title='Select path to add as a list')
        path = path.replace("/", os.sep)
        #si el path no pertany a la carpeta compartida amb mac o el path ja ha estat agregat com a llista sortim
        if (EXPORT_PATH not in path) | (path in path_s.values): 
            if EXPORT_PATH not in path: msg = "It's not possible to add a folder not shared with mac"
            if path in path_s.values: msg = "This path has already been added as a list"
            messagebox.showwarning(message=msg, title="Error")
            return
        
        time0 = time.time()
        folder_playlist_df = self.linker.build_loc_path_synched_df(path, subfolders=False)
        time1= time.time()
        
        if folder_playlist_df.empty:
            messagebox.showwarning(title="Error", message=f"No hi ha arxius d'audio a {path}.")
            return
        
        self.append_list_to_playlists(folder_playlist_df)
              
        self.local_list_selection_after_edit(event="add")
        
        lista = path.split(os.sep)[-1]
        messagebox.showinfo(title="Info", message=f"Afegit {lista} a LOCAL en {time1 - time0:.2f}s.")
        
        
    def delete_local_list(self):
        msg = f"Do you want to remove {self.lm.folder_list_selected[1]} list?"
        response = yesno_to_bool(messagebox.askquestion(message=msg, title="Question"))
        if not response: return
        
        remove_df = self.lm.selected_list_df
        self.remove_list_from_playlist(remove_df)
        self.save_playlists_df()
     
        #intentem seleccionar la última llista local (si nhi ha, sino no fara res)
        self.local_list_selection_after_edit(event="delete list")

    def reload_local_list(self):
        filepath = self.lm.selected_list_df.Location.iloc[0]
        path = os.sep.join(filepath.split(os.sep)[:-1])
        self.remove_list_from_playlist(self.lm.selected_list_df)

        time0 = time.time()
        folder_playlist_df = self.linker.build_loc_path_synched_df(path, subfolders=False)
        time1= time.time()


        self.infoselected_list_df = self.lm.selected_list_df
        self.infonewlist_df = folder_playlist_df
        
        
        
        
        
        
        self.append_list_to_playlists(folder_playlist_df)
        
              
        self.local_list_selection_after_edit(event="add")
        
        lista = path.split(os.sep)[-1]
        messagebox.showinfo(title="Info", message=f"Reloaded {lista} to LOCAL in {time1 - time0:.2f}s.")

            
    def delete_track(self, later=False):
        #si no hi ha cap track seleccionat retorna
        if not self.ct.loaded: return
        if self.ct.file_origin == "Itunes": return
        
        if not later: #delete forever
            msg = f"Do you want to delete {self.ct.pc_loc}?"
            response = yesno_to_bool(messagebox.askquestion(message=msg, title="Question"))
            if not response: return
        
        if self.media.audio.is_playing(): self.media.audio.stop()
        
        #podem agafar current en comptes de selected perquè el track s'ha de carregar obligatòriament
        list_df = self.lm.current_list_df 
        del_track_i = list_df[list_df.pc_loc == self.ct.pc_loc].index
        
        #esborrem el track a la nova llista i reubiquem positions
        new_list_df = list_df.drop(index=del_track_i)
        new_list_df["Position"] = range(len(new_list_df))
        new_list_df["Length"] = len(new_list_df)
        
        #esborrem la llista original de playlists_df i afegim la nova
        self.playlists_df.drop(index=list_df.index, inplace=True)
        self.append_list_to_playlists(new_list_df)
               
        if later: #si no es un delete movem el track a EXPORT_PATH
            filename = self.ct.pc_loc.split(os.sep)[-1]
            new_filepath = EXPORT_PATH + os.sep + filename
            os.replace(self.ct.pc_loc, new_filepath)
        else: os.remove(self.ct.pc_loc)
        
        self.local_list_selection_after_edit(event="delete track")

    
    def remove_list_from_playlist(self, list_df):
        del_index = list_df.index
        self.playlists_df.drop(index=del_index, inplace=True)
        self.playlists_df.reset_index(drop=True, inplace=True)

        
    def refresh_list_from_browser(self):   
        if hasattr(self.media, 'audio'):
            if self.media.audio.is_playing(): self.media.audio.stop()
            
        refresh_df = self.lm.selected_list_df.copy()
        refreshed_df, added, deleted = self.linker.build_synched_from_list_df(refresh_df)
        
        if added == deleted == 0: msg = "Sense canvis."
        else:
            # en el cas que estiguem reproduint la mateixa llista que refresquem descarreguem current track
            # perque no ens situi sobre de l'iid de l'audio reproduit anteriorment
            if self.lm.folder_list_current == self.lm.folder_list_selected: self.ct.loaded = False
            self.remove_list_from_playlist(refresh_df)
            self.append_list_to_playlists(refreshed_df)
            msg = f"{added} arxius nous vinculats.\n{deleted} arxius eliminats de la carpeta."
            self.local_list_selection_after_edit("refresh")
        
        messagebox.showinfo(title="Info", message=f"Llista {self.lm.folder_list_selected[1]} actualitzada:\n" + msg)
    
    def import_itunes_to_browser(self):
        xml_mod_date = remote_xml_conected_assert()
        with open('data.json') as f:
            data = json.load(f) 
        if data["DATA"]["XML_MODIFIED"] == xml_mod_date:
            msg = f"Les versions del XML coincideixen de data\n{xml_mod_date}\n"
            messagebox.showinfo(title="Info", message= msg + "Si hi ha informació nova a Itunes, desa la nova versió de la biblioteca.")
            return

        if hasattr(self.media, 'audio'):
            if self.media.audio.is_playing(): self.media.audio.stop()
        self.delete_audio_tree(lista=True, folder=True)#esborrem trees
        
        #mostrem label
        if self.loading_lbl.winfo_exists(): self.loading_lbl.destroy()
        self.loading_lbl = tk.Label(self, text="Reading remote iTunes library and loading playlists...",
                                    bg="white", fg="red", font=("Courier", 30), relief="solid", borderwidth=5)
        self.loading_lbl.place(relx=.5, rely=.5, anchor=tk.CENTER)
        self.update()
        
        #importem biblioteca i construim finder i recollim missatge d'importacio
        import_msg = self.import_itunes_library_and_linker()
                
        #desem tracks_df i itunes playlists
        self.tracks_df.to_csv("temp/tracks_df.csv", index=False)
        self.save_playlists_df()
        #desem finder_df --> ho fem a utilities, al thread de creació
        
        #desem la data de modificacio de l'XML importat
        data["DATA"]["XML_MODIFIED"] = remote_xml_conected_assert()
        with open('data.json', 'w') as f:
            json.dump(data, f)
                
        self.build_uniques()
          
        self.build_tree_folders()
        self.origin_route.set("Find remotes")

        if self.loading_lbl.winfo_exists(): 
            self.loading_lbl.destroy()
            messagebox.showinfo(title="Info", message=import_msg)
            self.list_btn1.grid_remove()
            self.list_btn2.grid_remove()
            #self.list_btn3.grid_remove()

    def create_playlist_to_export(self):
        list_df = self.lm.selected_list_df.copy()
        
        playlist_paths_l = list_df.apply(lambda x: x.mac_loc if x.linked else x.pc_mac_loc, axis=1).tolist()
        
        #mirem si hi ha playlist d'esborrats i els afegim a l'inici de l'export
        folder_path = os.sep.join(self.lm.selected_list_df.iloc[0]["Location"].split(os.sep)[0:-1])
        delete_path = folder_path + os.sep + "- ESBORRAR.m3u8"
        if os.path.exists(delete_path):
            to_delete = True
            with open(delete_path) as text_file:
                lines_l = text_file.readlines()
                delete_l = [f.replace("\n", "") for f in lines_l]
        else:
            to_delete, delete_l = False, []
        
        playlist_paths_l = delete_l + playlist_paths_l
        
        playlist = f"{self.lm._folder_list_selected[1]} (export).m3u8"
        playlist_path = EXPORT_PATH + os.sep + playlist
        
        with open(playlist_path, "w", encoding='utf-8') as text_file:
            text_file.write("\n".join(playlist_paths_l))
            
        del_msg = "\nAfegits tracks per esborrar a iTunes" if to_delete else ""  
        msg = f"\nAfegida a '{EXPORT_PATH}' la playlist '{playlist}'"
        messagebox.showinfo(title="Info", message=msg+del_msg)
        
    # def build_new_tracks_folder(self):
    #     msg = "Do you want to create a new folder with unlink tracks?"
    #     response = yesno_to_bool(messagebox.askquestion(message=msg, title="Question"))
    #     if not response: return
        
    #     list_df = self.lm.selected_list_df.copy()
        
    #     filepath0 = list_df.pc_loc.iloc[0]
    #     path = os.sep.join(filepath0.split(os.sep)[:-1])
    #     export_path = path + os.sep + "export"
        
    #     if not os.path.isdir(export_path): os.mkdir(export_path)
        
    #     cond = list_df.linked #els que no hem linkat son els exportables
    #     no_linked_df = list_df[~cond]
    #     paths_new = no_linked_df.pc_loc.tolist()
        
    #     for old_file_path in paths_new:
    #         filename = old_file_path.split(os.sep)[-1]
    #         new_file_path = export_path + os.sep + filename
    #         os.replace(old_file_path, new_file_path)
        
    #     self.refresh_list_from_browser(new_tracks_folder=True)
        
    #     #afegim el directori export com una llista (simulem un refresh del export)
    #     self.add_local_list(new_tracks_folder_path=export_path)
        
        
#FUNCIONS PER EDITAR TRACKS
    def upgrade_track(self):
        #agafem localitzacions del track que es mostra a itunes panel   
        mac_loc, mac_pc_loc = self.ct.itunes_link_loc

        if self.ct.linked: self.change_link(False, upgrade=True)

        playlist_path = os.sep.join(self.ct.pc_loc.split(os.sep)[0:-1]) + os.sep + "- ESBORRAR.m3u8"
        if os.path.exists(playlist_path):
            with open(playlist_path) as text_file:
                lines_l = text_file.readlines()
                files_l = [f.replace("\n", "") for f in lines_l]
        else: files_l = []

        if mac_loc not in files_l:
            files_l.append(mac_loc) 
            with open(playlist_path, "w", encoding='utf-8') as text_file:
                text_file.write("\n".join(files_l))

        self.editor.fill_editor_fields(mac_pc_loc)
        
        #afegim notacio de Esborrat al genere i creem una llista per actualitzar aquests tracks
        audio = AudioFile(mac_pc_loc)
        audio.genre = "!99 !BORRAT"
        audio.save()
            
         
    def change_link(self, linked, upgrade=False):
        if self.media.audio.is_playing(): self.media.audio.stop()
        
        index_track = self.playlists_df[self.playlists_df.Location == self.ct.Location].index
        self.playlists_df.loc[index_track, "linked"] = linked
        
        if linked:
            self.playlists_df.loc[index_track, "level"] = -2 if upgrade else 6
            self.playlists_df.loc[index_track, "color"] = "red"
            self.playlists_df.loc[index_track, "mac_pc_loc"] = self.ct.itunes_link_loc[1]
            self.playlists_df.loc[index_track, "mac_loc"] = self.ct.itunes_link_loc[0]
        else:
            self.playlists_df.loc[index_track, "level"] = -2 if upgrade else -1
            self.playlists_df.loc[index_track, "color"] = "darkgreen"

        self.save_playlists_df()
        self.list_selected("change link")
        if hasattr(self.itunes_tags, 'window'): self.itunes_tags.window.destroy()
        self.audio_selected(None)
        
    def add_hashtag_to_list(self):
        if self.lm.folder_list_selected[0] != "LOCAL": 
            messagebox.showwarning(message="It's not a local list", title="Warning")
            return
        list_df = self.lm.selected_list_df
        hashtag = tkinter.simpledialog.askstring(title="Add Hashtag", prompt="Hashtag to add all tracks list:")
        hashtag = "#" + hashtag
        
        def add_hashtag_to_pathfiles(hashtag, iterable):
            for trackpath in iterable:
                audio = AudioFile(trackpath)
                if not hashtag in audio.comments:
                    audio.comments = audio.comments + " " + hashtag
                    audio.comments = format_spaces(audio.comments)
                    audio.save()
        
        add_hashtag_to_pathfiles(hashtag, list_df.pc_loc)
                
        msg = "Do you want to add it to linked remote files?"
        remote_add = yesno_to_bool(messagebox.askquestion(message=msg, title="Question"))
        
        if remote_add: 
            trackpaths_l = []
            for row in list_df.itertuples():
                if row.linked: trackpaths_l.append(row.mac_pc_loc)
            
            add_hashtag_to_pathfiles(hashtag, trackpaths_l)
                
#     def del_hashtag_from_list(self):
#         #funció a la que passant un unique ens retorna els uniques sota un pattern
#         def uniques(column, pattern, genre=None):
#             if genre == None:
#                 data = it.tracks_df[column].unique()
#             else:
#                 data = it.tracks_df[it.tracks_df.Genre == genre]
#                 data = data[column].unique()
#             matches = []
#             for value in data:
#                 found = re.findall(pattern, value)
#                 matches = matches + found
#                 matches = set(matches)
#                 matches = list(matches)
#             return matches
        
        
#         #hashtags unics
#         pattern = r"\#\S+"
#         data = uniques("Comments", pattern)
#         data.sort()
#         data
        
#----------------------------------NAVIGATOR FUNCTIONS-------------------------------------
    def delete_audio_tree(self, lista=False, folder=False):
        for song in self.audio_list.get_children():
            self.audio_list.delete(song)
        if not lista: return
        for lista in self.list_list.get_children():
            self.list_list.delete(lista)
        if not folder: return
        for folder in self.folder_list.get_children():
            self.folder_list.delete(folder)
            
    def remove_all_widgets(self):
        self.remove_list_btn.grid_remove()
        self.reload_list_btn.grid_remove()
        self.update_list_btn.grid_remove()
        self.sort_cb.grid_remove()
        self.origin_route_cb.grid_remove()
        self.list_btn1.grid_remove()
        self.list_btn2.grid_remove()
        #self.list_btn3.grid_remove()
# ----------
    def build_tree_folders(self):
        folders = self.playlists_df.Folder.unique()
        #omplim llista folders
        for folder, folder_iid in zip(folders, range(len(folders))):
            if folder == "LOCAL": folder_type = "Local"
            else: folder_type = "Remote"
            self.folder_list.insert("", tk.END, text=folder, iid=folder_iid, tags=("selected", folder_type))
        self.folder_list.tag_configure('Local', foreground='red')
        self.folder_list.tag_configure('Remote', foreground='black')

    def folder_selected(self, event):
        self.delete_audio_tree(lista=True)
        self.remove_all_widgets()
        
        folder_iid = self.folder_list.selection()[0] #treiem l'iid del que sha seleccionat
        folder = self.folder_list.item(folder_iid, option="text")
        folder_df = self.playlists_df[self.playlists_df.Folder == folder]
        lists = folder_df.List.unique()
        #omplim la llista de llistes
        for lista, list_iid in zip(lists, range(len(lists))):
            if self.playlists_df[self.playlists_df.List == lista].Smart.iat[0]:
                list_type = "Smart"
            else: list_type = "Manual"
            self.list_list.insert("", tk.END, iid=list_iid, text=lista, 
                                  tags=("selected", list_type), values=(folder_iid, folder))
        self.list_list.tag_configure('Smart', foreground='blue')
        self.list_list.tag_configure('Manual', foreground='black')
    
    def list_selected(self, event):
        if not self.enable_trace_vars:return #desabilitem binds
        #si no hi ha cap llista seleccionada retorna
        if not self.list_list.selection(): return
        
        self.delete_audio_tree()
        self.remove_all_widgets()
        
        folder, lista  = self.return_folder_list_selected(return_ids=False)
        
        if type(event) == tk.Event:
            if event.widget.widgetName == 'ttk::treeview':
                self.lm.folder_list_selected = [folder, lista]
            elif event.widget.widgetName == 'ttk::combobox':
                self.lm.sorted_by_cb()
                #if self.ct.loaded: self.ct.update() #NO SE SI AQUI ESTIC REDUNDANT L'UPDATE
        else:
            #en el cas que venim d'esborrar track o de refrescar hem de rellegir la llista seleccionada
            if event in ["delete track", "refresh", "change link"]: self.lm.update_selected_list()
            #en el cas de venir d'un afegiment de llista local, o eliminació, ho tractem som una selecció al treeview
            if event in ["add", "delete list"]: self.lm.folder_list_selected = [folder, lista]
            
        list_df = self.lm.selected_list_df

        #si estem a la carpeta "LOCAL" mostrem widgets o sinó sols el d'ordenar
        if folder == "LOCAL":
            self.remove_list_btn.grid(row=0, column=1, padx=2)
            self.reload_list_btn.grid(row=0, column=2, padx=2)
            self.update_list_btn.grid(row=0, column=3, padx=2)
            self.origin_route_cb.grid(row=0, column=2, padx=2)
            self.list_btn1.grid(row=0, column=3, padx=2)
            self.list_btn2.grid(row=0, column=4, padx=2)
            #self.list_btn3.grid(row=0, column=5, padx=2)
        self.sort_cb.grid(row=0, column=0, padx=2)
        
        #omplim audio_list
        artist_s, title_s, level_s = list_df.Artist, list_df.Name, list_df.level
        index_s, location_s, color_s = pd.Series(range(len(list_df))), list_df.Location, list_df.color
        series_zip = zip(artist_s, title_s, level_s, index_s, location_s, color_s)
        for artist, title, level, audio_iid, location, color in series_zip:
            if folder == "LOCAL": level = int(level)
            else: level = ""
            self.audio_list.insert("", tk.END, iid=audio_iid, text=str(audio_iid+1), tags=("selected", color),
                                   values=(artist, title, level, folder, lista, location))
        
        for color in color_s.unique().tolist():
            self.audio_list.tag_configure(color, foreground=color)
        
        self.audio_list.yview_moveto(0)
        
        #si estem al mateix folder/list que el track carregat es selecciona i mostrem amb see() (a audio selected)
        if self.ct.loaded:
            cond1 = self.folder_list.selection()[0] == self.ct.folder_iid
            cond2 = self.list_list.selection()[0] == self.ct.list_iid
            if cond1 & cond2:
                #print("estem al mateix folder/list que el track carregat")
                self.audio_list.selection_set(self.ct.audio_iid)
                
    def audio_selected(self, event):
        audio_iid = self.audio_list.selection()[0]
        v = self.audio_list.item(audio_iid, option="values")
        artist, title, level, folder, lista, location = v[0], v[1], v[2], v[3], v[4], v[5]
        current_track = CurrentTrack(self, folder, lista, location)
        playing_path = current_track.playing_path
        
        #en el cas q seleccionem el mateix track que s'esta reproduint no l'aturarem ni generarem nova instancia
        #a no ser que vinguem de linkar
        if self.ct.loaded:
            if self.media.audio.is_playing():
                if self.media.song_path == playing_path: return
                else: self.media.audio.stop() #si seleccionem qualsevol altre track aturem
        
        #check if file exists
        if not os.path.isfile(playing_path):
            if folder == "LOCAL":
                self.origin_route.set("Local mode")
                msg = f"No connection to {playing_path}.\n'Local mode' work mode changed"
            else: msg = f"No connection to {playing_path}."
            messagebox.showinfo(title="Info", message=msg)    
            return
                
        self.ct = current_track

        #omplim title_song
        self.title_song.set(self.ct.Artist + " - " + self.ct.Name)
        #si treballem amb tot locals establirem el color del title_song en vermell en el cas que l'edició sigui inutil
        if self.ct.linked & (EXPORT_PATH in self.ct.playing_path): #(linkats quan treballem en 'Local mode')
            self.title_song_lbl.configure(fg="red")
        else: self.title_song_lbl.configure(fg="blue")
        
        #omplim etiqueta de ruta
        position = str(int(self.ct.audio_iid) + 1)
        self.route_song.set(folder + "\n" + lista + "\n" +  " (" + position + ")")
            
        self.itunes_tags.fill_itunes_fields()
        self.reader.fill_audio_fields()
        
        if self.reader.autoload.get():
            self.editor.fill_editor_fields()
        
        #fem visible el track en reproducció
        self.audio_list.see(self.ct.audio_iid)
        #carreguem l'art
        image = extract_pic_from_metadata(playing_path, size=(311,311))
        self.label_art.config(image=image)
        self.label_art.image=image
        
        #tanquem la finestra links si esta oberta
        if hasattr(self.itunes_tags, 'window'): self.itunes_tags.window.destroy()
        
        #copiem el artist - title al clipboard
        self.clipboard_clear()
        first_artist = self.ct.Artist.split(";")[0]
        self.clipboard_append(first_artist + " " + self.ct.Name)
        
        #enviem ordre de reproduir al final per deixar temps si hi ha algun thread que ens penja el soft
        self.play_path(playing_path)


        
class CurrentTrack():
    def __init__(self, root, folder=None, lista=None, location=None, loaded=True):
        self.root = root
        self._loaded = loaded
        if not self._loaded: return
        
        self.Folder, self.List = folder, lista
        self.Location = location #location és la ubicacio desde pc
        
        if folder == "LOCAL": self.file_origin = "Local"
        else: self.file_origin = "Itunes"
        
        #quan inicialitzem ct s'encarrega de dirli al list manager que la seleccionada es current
        self.root.lm.set_selected_as_current()
        self._fill_track_attributes()
        
        #Location - Ubicació, desde pc local, de l'arxiu amb el que treballarem 
        #           si és d'itunes, ubicació al mac, si es LOCAL ubicació al EXPORT_PATH
        #playing_path - Ubicació de current track (pot ser un LOCAL linkat, local sense linkar, o remot)
    
    @property
    def loaded(self):
        return self._loaded

    @loaded.setter
    def loaded(self, loaded):
        if self._loaded == loaded: return
        self._loaded = loaded
        if not self._loaded: 
            self._delete_track_attributes()
            
    def _fill_track_attributes(self):
        cond = self.root.lm.current_list_df.Location == self.Location
        track_df = self.root.lm.current_list_df[cond]
        self._track_dic = track_df.iloc[0].to_dict()
        
        for attr, value in self._track_dic.items():
            setattr(self, attr, value)
        
        #aquesta variable referencia el track que es mostra a l'itunes panel. Pot no ser el de major level, i aleshores no coincidira
        #amb self.mac_loc, self.mac_pc_loc, però quan sinicialitza és per visualitzar el de major level (linkat o no)
        self.itunes_link_loc = [self.mac_loc, self.mac_pc_loc]
        if self.linkable == False: self.linked = False
        
        #si duration es integer(seconds) passem a string
        if type(self.Duration) == int: self.Duration = ms_to_string(self.Duration, metric="seconds")
                
        #si tenim seleccionat l'opció local carreguem la ruta local dels audios, sino la linkada, si n'hi ha
        if (self.file_origin == "Local") & (self.root.origin_route.get() == "Find remotes"):
            self.playing_path = self.mac_pc_loc if self.linked else self.pc_loc
        else: self.playing_path = self.Location #location és ubicacio desde pc (pot estar a mac o a pc)
        
        #posem un playing_origin per si la ruta del paying_path es local o remota d'itunes
        self.playing_origin = "Local" if EXPORT_PATH in self.playing_path else "Itunes"
                
    def update(self):
        self.current_list_df = self.root.lm.selected_list_df
        self._fill_track_attributes()
        
    def _delete_track_attributes(self):
        attributes = list(self.__dict__.keys())
        for attr in attributes:
            delattr(self, attr)
        self._loaded = False
        
              
              
class ListManager():
    def __init__(self, root):
        self.root = root
        self._folder_list_selected = ["Folder", "List"]
        self.folder_list_current = self._folder_list_selected
        self.current_sort_cb = self.root.sortby_cb.get()

    @property
    def folder_list_selected(self):
        return self._folder_list_selected
    
    #llista seleccionada cridada modificant l'atribut per treelist
    #modifican la list selected en el cas que sigui diferent a lanterior seleccionada
    @folder_list_selected.setter
    def folder_list_selected(self, folder_list_selected):
        if folder_list_selected == self._folder_list_selected: return #no hem canviat de llista seleccionada
        self._folder_list_selected = folder_list_selected
        if self._folder_list_selected == self.folder_list_current: #hem tornat a current llista
            self.root.enable_trace_vars = False
            self.root.sortby_cb.set(self.current_sort_cb)
            self.root.enable_trace_vars = True
        else:
            sort_by = self.default_list_sort(folder_list_selected)
            self.root.sortby_cb.set(sort_by)
            #print("hem ordenat la llista per", sort_by)
        self.selected_list_df = self.return_list_df(folder_list=folder_list_selected, sort=True, ids=True)    

    
    #si hem seleccionat un track de la llista seleccionada actual, copia selected_list a current_list
    def set_selected_as_current(self):
        #print("cambiem current list a", self._folder_list_selected)
        self.folder_list_current = self._folder_list_selected
        self.current_sort_cb = self.root.sortby_cb.get()
        self.current_list_df = self.selected_list_df.copy()
            
    #si hem fet un delete track (la llista seleccionada es la mateixa q lactual) 
    def update_selected_list(self):
        self.selected_list_df = self.return_list_df(folder_list=self._folder_list_selected, sort=True, ids=True)
        
    #hem cridat a list_select seleccionant valor al combobox d'ordenacio (la llista seleccionada es la mateixa q lactual)      
    def sorted_by_cb(self):
        self.selected_list_df = self.return_list_df(list_df=self.selected_list_df, sort=True, ids=True)
        #si estem ordenant la llista que sona (current) copiem l'ordenada a current i actualitzem el track
        if self._folder_list_selected == self.folder_list_current:
            self.current_list_df = self.selected_list_df.copy()
            self.current_sort_cb = self.root.sortby_cb.get()
            self.root.ct.update()
    
    #torna la ordenació per defecte de qualsevol llista
    def default_list_sort(self, folder_list_selected):
        if folder_list_selected[0] == "LOCAL":
            list_df = self.return_list_df(folder_list=folder_list_selected)
            if list_df.iloc[0]["Smart"]: return "Position"
        return "Creation (desc.)"
        
    def return_list_df(self, folder_list=None, list_df=None, sort=False, ids=False):
        playlists_df = self.root.playlists_df
        if folder_list != None: 
            list_df_passed = False
            folder, lista = folder_list
            cond1 = playlists_df.Folder == folder
            cond2 = playlists_df.List == lista
            list_df = playlists_df[cond1 & cond2].copy()
        else: list_df_passed = True
        
        if sort:
            sortby, ascending = self.root.sortby_cb.get(), True
            if any([x in self.root.sortby_cb.get() for x in ["Creation", "Modified"]]):
                if "Creation" in self.root.sortby_cb.get(): sortby = "Creation"
                else: sortby = "Date Modified"
                if "(desc.)" in self.root.sortby_cb.get(): ascending = False
            list_df = list_df.sort_values(by=[sortby, "Name"], ascending=[ascending, True])

        if ids:
            if list_df_passed:
                folder = list_df.iloc[0]["Folder"]
                lista = list_df.iloc[0]["List"]
                cond1 = playlists_df.Folder == folder
                cond2 = playlists_df.List == lista

            folders_l = playlists_df.Folder.unique().tolist()
            folder_iid = str(folders_l.index(folder))

            folder_df = playlists_df[cond1]
            lists_l = folder_df.List.unique().tolist()
            list_iid = str(lists_l.index(lista))
       
            #afegim ids
            list_df["folder_iid"] = folder_iid
            list_df["list_iid"] = list_iid
            list_df["audio_iid"] = [str(x) for x in range(len(list_df))]
         
        return list_df   

   

   



app = Interface(it=None)
app.mainloop()

In [9]:
import pandas as pd
import math

# Ejemplo de dataframes
data_a = {'clave1': [1, 2, 3],
          'clave2': ['A', 'B', 'C'],
          'col1': [10, 20, 30],
          'col2': [40, 50, 60]}

data_b = {'clave1': [1, 2, 4, 2],
          'clave2': ['X', 'B', 'Z', 'B'],
          'col1': [100, 200, 300, 255],
          'col2': [400, 500, 600, 444]}

df_a = pd.DataFrame(data_a)
df_b = pd.DataFrame(data_b)


df_a, df_b

(   clave1 clave2  col1  col2
 0       1      A    10    40
 1       2      B    20    50
 2       3      C    30    60,
    clave1 clave2  col1  col2
 0       1      X   100   400
 1       2      B   200   500
 2       4      Z   300   600
 3       2      B   255   444)

In [10]:
claves = ['clave1', 'clave2']
cols = df_a.columns.to_list()
for x in claves: cols.remove(x)
cols

['col1', 'col2']

In [11]:
df = pd.merge(df_a, df_b, on=['clave1', 'clave2'], how='left')
# Crear un diccionario dinámicamente
nuevas_cols = {col + "_y": col for col in cols}
# Utilizar la función rename
df.rename(columns=nuevas_cols, inplace=True)
df

,clave1,clave2,col1_x,col2_x,col1,col2
0,1,A,10,40,NaN,NaN
1,2,B,20,50,200.0,500.0
2,2,B,20,50,255.0,444.0
3,3,C,30,60,NaN,NaN


In [14]:
def custom_function(row):
    for col in cols:
        if math.isnan(row[col]): row[col] = row[col + "_x"] 
    return row

df = df.apply(custom_function, axis=1)

In [16]:
df = df.filter(regex='^(?!.*_x$)')

,clave1,clave2,col1,col2
0,1,A,10.0,40.0
1,2,B,200.0,500.0
2,2,B,255.0,444.0
3,3,C,30.0,60.0


In [2]:
app.infonewlist_df

,Folder,List,Artist,Name,Duration,Creation,pc_loc,mac_pc_loc,mac_loc,level,linkable,color,title_song_color,pc_mac_loc,Position,Length,Smart,Codec,Location,linked
0,LOCAL,TOP 60 DEL 2023,Nation of Language,Sole Obsession,4:17,2023-12-29 18:49:20,D:\MUSIQUE\PASAR A MAC\TOP 60 DEL 2023\Nation ...,\\169.254.170.1\Musica\Itunes Media\Nation of ...,file:///Users/legoteque/Music/Itunes%20Media/N...,5,True,red,red,/Volumes/PASAR A MAC/TOP 60 DEL 2023/Nation of...,0,60,True,MP3,D:\MUSIQUE\PASAR A MAC\TOP 60 DEL 2023\Nation ...,True
1,LOCAL,TOP 60 DEL 2023,Blur,Barbaric,4:09,2023-12-29 18:49:20,D:\MUSIQUE\PASAR A MAC\TOP 60 DEL 2023\Blur - ...,\\169.254.170.1\Musica\Itunes Media\Blur - Bar...,file:///Users/legoteque/Music/Itunes%20Media/B...,5,True,red,red,/Volumes/PASAR A MAC/TOP 60 DEL 2023/Blur - Ba...,1,60,True,MP3,D:\MUSIQUE\PASAR A MAC\TOP 60 DEL 2023\Blur - ...,True
2,LOCAL,TOP 60 DEL 2023,Spirits Of Saturn,Feelings Agency,6:10,2023-12-29 18:49:21,D:\MUSIQUE\PASAR A MAC\TOP 60 DEL 2023\Spirits...,\\169.254.170.1\Musica\Itunes Media\Spirits Of...,file:///Users/legoteque/Music/Itunes%20Media/S...,5,True,red,red,/Volumes/PASAR A MAC/TOP 60 DEL 2023/Spirits O...,2,60,True,MP3,D:\MUSIQUE\PASAR A MAC\TOP 60 DEL 2023\Spirits...,True
3,LOCAL,TOP 60 DEL 2023,Peace,Happy Cars,3:18,2023-12-29 18:49:29,D:\MUSIQUE\PASAR A MAC\TOP 60 DEL 2023\Peace -...,\\169.254.170.1\Musica\Itunes Media\Peace - Ha...,file:///Users/legoteque/Music/Itunes%20Media/P...,5,True,red,red,/Volumes/PASAR A MAC/TOP 60 DEL 2023/Peace - H...,3,60,True,MP3,D:\MUSIQUE\PASAR A MAC\TOP 60 DEL 2023\Peace -...,True
4,LOCAL,TOP 60 DEL 2023,English Teacher,Nearly Daffodils,3:45,2023-12-29 18:49:37,D:\MUSIQUE\PASAR A MAC\TOP 60 DEL 2023\English...,\\169.254.170.1\Musica\Itunes Media\English Te...,file:///Users/legoteque/Music/Itunes%20Media/E...,5,True,red,red,/Volumes/PASAR A MAC/TOP 60 DEL 2023/English T...,4,60,True,MP3,D:\MUSIQUE\PASAR A MAC\TOP 60 DEL 2023\English...,True
5,LOCAL,TOP 60 DEL 2023,Depeche Mode,Ghosts Again,3:59,2023-12-29 18:49:50,D:\MUSIQUE\PASAR A MAC\TOP 60 DEL 2023\Depeche...,\\169.254.170.1\Musica\Itunes Media\Depeche Mo...,file:///Users/legoteque/Music/Itunes%20Media/D...,5,True,red,red,/Volumes/PASAR A MAC/TOP 60 DEL 2023/Depeche M...,5,60,True,MP3,D:\MUSIQUE\PASAR A MAC\TOP 60 DEL 2023\Depeche...,True
6,LOCAL,TOP 60 DEL 2023,The Blaze,LONELY,4:25,2023-12-29 18:49:51,D:\MUSIQUE\PASAR A MAC\TOP 60 DEL 2023\The Bla...,\\169.254.170.1\Musica\Itunes Media\The Blaze ...,file:///Users/legoteque/Music/Itunes%20Media/T...,5,True,red,red,/Volumes/PASAR A MAC/TOP 60 DEL 2023/The Blaze...,6,60,True,MP3,D:\MUSIQUE\PASAR A MAC\TOP 60 DEL 2023\The Bla...,True
7,LOCAL,TOP 60 DEL 2023,Melenas,Bang,5:10,2023-12-29 18:50:02,D:\MUSIQUE\PASAR A MAC\TOP 60 DEL 2023\Melenas...,\\169.254.170.1\Musica\Itunes Media\Melenas - ...,file:///Users/legoteque/Music/Itunes%20Media/M...,5,True,red,red,/Volumes/PASAR A MAC/TOP 60 DEL 2023/Melenas -...,7,60,True,MP3,D:\MUSIQUE\PASAR A MAC\TOP 60 DEL 2023\Melenas...,True
8,LOCAL,TOP 60 DEL 2023,Water From Your Eyes,Barley,3:29,2023-12-29 18:50:04,D:\MUSIQUE\PASAR A MAC\TOP 60 DEL 2023\Water F...,\\169.254.170.1\Musica\Itunes Media\Water From...,file:///Users/legoteque/Music/Itunes%20Media/W...,5,True,red,red,/Volumes/PASAR A MAC/TOP 60 DEL 2023/Water Fro...,8,60,True,MP3,D:\MUSIQUE\PASAR A MAC\TOP 60 DEL 2023\Water F...,True
9,LOCAL,TOP 60 DEL 2023,DAIISTAR,Parallel,2:55,2023-12-29 18:50:17,D:\MUSIQUE\PASAR A MAC\TOP 60 DEL 2023\DAIISTA...,\\169.254.170.1\Musica\Itunes Media\DAIISTAR -...,file:///Users/legoteque/Music/Itunes%20Media/D...,5,True,red,red,/Volumes/PASAR A MAC/TOP 60 DEL 2023/DAIISTAR ...,9,60,True,MP3,D:\MUSIQUE\PASAR A MAC\TOP 60 DEL 2023\DAIISTA...,True


['Folder',
 'List',
 'Duration',
 'Creation',
 'pc_loc',
 'mac_pc_loc',
 'mac_loc',
 'level',
 'linkable',
 'color',
 'title_song_color',
 'pc_mac_loc',
 'Position',
 'Length',
 'Smart',
 'Codec',
 'Location',
 'linked']

In [22]:
cols = app.infonewlist_df.columns.to_list()
for x in ["Artist", "Name"]: cols.remove(x)
pd.merge(app.infonewlist_df, app.infoselected_list_df, how="left", on=["Artist", "Name"])

,Folder_x,List_x,Artist,Name,Duration_x,Creation_x,pc_loc_x,mac_pc_loc_x,mac_loc_x,level_x,linkable_x,color_x,title_song_color_x,pc_mac_loc_x,Position_x,Length_x,Smart_x,Codec_x,Location_x,linked_x,Folder_y,List_y,Smart_y,Length_y,Position_y,Duration_y,Codec_y,Creation_y,Date Modified,Location_y,pc_loc_y,mac_loc_y,pc_mac_loc_y,linkable_y,linked_y,mac_pc_loc_y,level_y,color_y,title_song_color_y,folder_iid,list_iid,audio_iid
0,LOCAL,TOP 60 DEL 2023,Nation of Language,Sole Obsession,4:17,2023-12-29 18:49:20,D:\MUSIQUE\PASAR A MAC\TOP 60 DEL 2023\Nation ...,\\169.254.170.1\Musica\Itunes Media\Nation of ...,file:///Users/legoteque/Music/Itunes%20Media/N...,5,True,red,red,/Volumes/PASAR A MAC/TOP 60 DEL 2023/Nation of...,0,60,True,MP3,D:\MUSIQUE\PASAR A MAC\TOP 60 DEL 2023\Nation ...,True,LOCAL,TOP 60 DEL 2023,True,60,0.0,4:17,MP3,2023-12-29 18:49:20,,D:\MUSIQUE\PASAR A MAC\TOP 60 DEL 2023\Nation ...,D:\MUSIQUE\PASAR A MAC\TOP 60 DEL 2023\Nation ...,file:///Users/legoteque/Music/Itunes%20Media/N...,/Volumes/PASAR A MAC/TOP 60 DEL 2023/Nation of...,True,True,\\169.254.170.1\Musica\Itunes Media\Nation of ...,5.0,red,red,0,9,0
1,LOCAL,TOP 60 DEL 2023,Blur,Barbaric,4:09,2023-12-29 18:49:20,D:\MUSIQUE\PASAR A MAC\TOP 60 DEL 2023\Blur - ...,\\169.254.170.1\Musica\Itunes Media\Blur - Bar...,file:///Users/legoteque/Music/Itunes%20Media/B...,5,True,red,red,/Volumes/PASAR A MAC/TOP 60 DEL 2023/Blur - Ba...,1,60,True,MP3,D:\MUSIQUE\PASAR A MAC\TOP 60 DEL 2023\Blur - ...,True,LOCAL,TOP 60 DEL 2023,True,60,1.0,4:09,MP3,2023-12-29 18:49:20,,D:\MUSIQUE\PASAR A MAC\TOP 60 DEL 2023\Blur - ...,D:\MUSIQUE\PASAR A MAC\TOP 60 DEL 2023\Blur - ...,file:///Users/legoteque/Music/Itunes%20Media/B...,/Volumes/PASAR A MAC/TOP 60 DEL 2023/Blur - Ba...,True,True,\\169.254.170.1\Musica\Itunes Media\Blur - Bar...,5.0,red,red,0,9,1
2,LOCAL,TOP 60 DEL 2023,Spirits Of Saturn,Feelings Agency,6:10,2023-12-29 18:49:21,D:\MUSIQUE\PASAR A MAC\TOP 60 DEL 2023\Spirits...,\\169.254.170.1\Musica\Itunes Media\Spirits Of...,file:///Users/legoteque/Music/Itunes%20Media/S...,5,True,red,red,/Volumes/PASAR A MAC/TOP 60 DEL 2023/Spirits O...,2,60,True,MP3,D:\MUSIQUE\PASAR A MAC\TOP 60 DEL 2023\Spirits...,True,LOCAL,TOP 60 DEL 2023,True,60,2.0,6:10,MP3,2023-12-29 18:49:21,,D:\MUSIQUE\PASAR A MAC\TOP 60 DEL 2023\Spirits...,D:\MUSIQUE\PASAR A MAC\TOP 60 DEL 2023\Spirits...,file:///Users/legoteque/Music/Itunes%20Media/S...,/Volumes/PASAR A MAC/TOP 60 DEL 2023/Spirits O...,True,True,\\169.254.170.1\Musica\Itunes Media\Spirits Of...,5.0,red,red,0,9,2
3,LOCAL,TOP 60 DEL 2023,Peace,Happy Cars,3:18,2023-12-29 18:49:29,D:\MUSIQUE\PASAR A MAC\TOP 60 DEL 2023\Peace -...,\\169.254.170.1\Musica\Itunes Media\Peace - Ha...,file:///Users/legoteque/Music/Itunes%20Media/P...,5,True,red,red,/Volumes/PASAR A MAC/TOP 60 DEL 2023/Peace - H...,3,60,True,MP3,D:\MUSIQUE\PASAR A MAC\TOP 60 DEL 2023\Peace -...,True,LOCAL,TOP 60 DEL 2023,True,60,3.0,3:18,MP3,2023-12-29 18:49:29,,D:\MUSIQUE\PASAR A MAC\TOP 60 DEL 2023\Peace -...,D:\MUSIQUE\PASAR A MAC\TOP 60 DEL 2023\Peace -...,file:///Users/legoteque/Music/Itunes%20Media/P...,/Volumes/PASAR A MAC/TOP 60 DEL 2023/Peace - H...,True,True,\\169.254.170.1\Musica\Itunes Media\Peace - Ha...,5.0,red,red,0,9,3
4,LOCAL,TOP 60 DEL 2023,English Teacher,Nearly Daffodils,3:45,2023-12-29 18:49:37,D:\MUSIQUE\PASAR A MAC\TOP 60 DEL 2023\English...,\\169.254.170.1\Musica\Itunes Media\English Te...,file:///Users/legoteque/Music/Itunes%20Media/E...,5,True,red,red,/Volumes/PASAR A MAC/TOP 60 DEL 2023/English T...,4,60,True,MP3,D:\MUSIQUE\PASAR A MAC\TOP 60 DEL 2023\English...,True,LOCAL,TOP 60 DEL 2023,True,60,4.0,3:45,MP3,2023-12-29 18:49:37,,D:\MUSIQUE\PASAR A MAC\TOP 60 DEL 2023\English...,D:\MUSIQUE\PASAR A MAC\TOP 60 DEL 2023\English...,file:///Users/legoteque/Music/Itunes%20Media/E...,/Volumes/PASAR A MAC/TOP 60 DEL 2023/English T...,True,True,\\169.254.170.1\Musica\Itunes Media\English Te...,5.0,red,red,0,9,4
5,LOCAL,TOP 60 DEL 2023,Depeche Mode,Ghosts Again,3:59,2023-12-29 

In [3]:
app.infoselected_list_df

,Folder,List,Smart,Length,Position,Artist,Name,Duration,Codec,Creation,Date Modified,Location,pc_loc,mac_loc,pc_mac_loc,linkable,linked,mac_pc_loc,level,color,title_song_color,folder_iid,list_iid,audio_iid
1229,LOCAL,TOP 60 DEL 2023,True,60,0.0,Nation of Language,Sole Obsession,4:17,MP3,2023-12-29 18:49:20,,D:\MUSIQUE\PASAR A MAC\TOP 60 DEL 2023\Nation ...,D:\MUSIQUE\PASAR A MAC\TOP 60 DEL 2023\Nation ...,file:///Users/legoteque/Music/Itunes%20Media/N...,/Volumes/PASAR A MAC/TOP 60 DEL 2023/Nation of...,True,True,\\169.254.170.1\Musica\Itunes Media\Nation of ...,5.0,red,red,0,9,0
1230,LOCAL,TOP 60 DEL 2023,True,60,1.0,Blur,Barbaric,4:09,MP3,2023-12-29 18:49:20,,D:\MUSIQUE\PASAR A MAC\TOP 60 DEL 2023\Blur - ...,D:\MUSIQUE\PASAR A MAC\TOP 60 DEL 2023\Blur - ...,file:///Users/legoteque/Music/Itunes%20Media/B...,/Volumes/PASAR A MAC/TOP 60 DEL 2023/Blur - Ba...,True,True,\\169.254.170.1\Musica\Itunes Media\Blur - Bar...,5.0,red,red,0,9,1
1231,LOCAL,TOP 60 DEL 2023,True,60,2.0,Spirits Of Saturn,Feelings Agency,6:10,MP3,2023-12-29 18:49:21,,D:\MUSIQUE\PASAR A MAC\TOP 60 DEL 2023\Spirits...,D:\MUSIQUE\PASAR A MAC\TOP 60 DEL 2023\Spirits...,file:///Users/legoteque/Music/Itunes%20Media/S...,/Volumes/PASAR A MAC/TOP 60 DEL 2023/Spirits O...,True,True,\\169.254.170.1\Musica\Itunes Media\Spirits Of...,5.0,red,red,0,9,2
1232,LOCAL,TOP 60 DEL 2023,True,60,3.0,Peace,Happy Cars,3:18,MP3,2023-12-29 18:49:29,,D:\MUSIQUE\PASAR A MAC\TOP 60 DEL 2023\Peace -...,D:\MUSIQUE\PASAR A MAC\TOP 60 DEL 2023\Peace -...,file:///Users/legoteque/Music/Itunes%20Media/P...,/Volumes/PASAR A MAC/TOP 60 DEL 2023/Peace - H...,True,True,\\169.254.170.1\Musica\Itunes Media\Peace - Ha...,5.0,red,red,0,9,3
1233,LOCAL,TOP 60 DEL 2023,True,60,4.0,English Teacher,Nearly Daffodils,3:45,MP3,2023-12-29 18:49:37,,D:\MUSIQUE\PASAR A MAC\TOP 60 DEL 2023\English...,D:\MUSIQUE\PASAR A MAC\TOP 60 DEL 2023\English...,file:///Users/legoteque/Music/Itunes%20Media/E...,/Volumes/PASAR A MAC/TOP 60 DEL 2023/English T...,True,True,\\169.254.170.1\Musica\Itunes Media\English Te...,5.0,red,red,0,9,4
1234,LOCAL,TOP 60 DEL 2023,True,60,5.0,Depeche Mode,Ghosts Again,3:59,MP3,2023-12-29 18:49:50,,D:\MUSIQUE\PASAR A MAC\TOP 60 DEL 2023\Depeche...,D:\MUSIQUE\PASAR A MAC\TOP 60 DEL 2023\Depeche...,file:///Users/legoteque/Music/Itunes%20Media/D...,/Volumes/PASAR A MAC/TOP 60 DEL 2023/Depeche M...,True,True,\\169.254.170.1\Musica\Itunes Media\Depeche Mo...,5.0,red,red,0,9,5
1235,LOCAL,TOP 60 DEL 2023,True,60,6.0,The Blaze,LONELY,4:25,MP3,2023-12-29 18:49:51,,D:\MUSIQUE\PASAR A MAC\TOP 60 DEL 2023\The Bla...,D:\MUSIQUE\PASAR A MAC\TOP 60 DEL 2023\The Bla...,file:///Users/legoteque/Music/Itunes%20Media/T...,/Volumes/PASAR A MAC/TOP 60 DEL 2023/The Blaze...,True,True,\\169.254.170.1\Musica\Itunes Media\The Blaze ...,5.0,red,red,0,9,6
1236,LOCAL,TOP 60 DEL 2023,True,60,7.0,Melenas,Bang,5:10,MP3,2023-12-29 18:50:02,,D:\MUSIQUE\PASAR A MAC\TOP 60 DEL 2023\Melenas...,D:\MUSIQUE\PASAR A MAC\TOP 60 DEL 2023\Melenas...,file:///Users/legoteque/Music/Itunes%20Media/M...,/Volumes/PASAR A MAC/TOP 60 DEL 2023/Melenas -...,True,True,\\169.254.170.1\Musica\Itunes Media\Melenas - ...,5.0,red,red,0,9,7
1237,LOCAL,TOP 60 DEL 2023,True,60,8.0,Water From Your Eyes,Barley,3:29,MP3,2023-12-29 18:50:04,,D:\MUSIQUE\PASAR A MAC\TOP 60 DEL 2023\Water F...,D:\MUSIQUE\PASAR A MAC\TOP 60 DEL 2023\Water F...,file:///Users/legoteque/Music/Itunes%20Media/W...,/Volumes/PASAR A MAC/TOP 60 DEL 2023/Water Fro...,True,True,\\169.254.170.1\Musica\Itunes Media\Water From...,5.0,red,red,0,9,8
1238,LOCAL,TOP 60 DEL 2023,True,60,9.0,DAIISTAR,Parallel,2:55,MP3,2023-12-29 18:50:17,,D:\MUSIQUE\PASAR A MAC\TOP 60 DEL 2023\DAIISTA...,D:\MUSIQUE\PASAR A MAC\TOP 60 DEL 2023\DAIISTA...,file:///Users/legoteque/Music/Itunes%20Media/D...,/Volumes/PASAR A MAC/TOP 60 DEL 2023/DAIISTAR ...,True,True,\\169.254.170.1\Musica\Itunes Media\DAIISTAR -...,5.0,red,red,0,9,9


In [22]:
app.playlists_df[app.playlists_df.List == app.infopath.split(os.sep)[-1]]

,Folder,List,Smart,Length,Position,Artist,Name,Duration,Codec,Creation,Date Modified,Location,pc_loc,mac_loc,pc_mac_loc,linkable,linked,mac_pc_loc,level,color,title_song_color
1229,LOCAL,TOP 60 DEL 2023,True,60,0.0,Nation of Language,Sole Obsession,4:17,MP3,2023-12-29 18:49:20,,D:\MUSIQUE\PASAR A MAC\TOP 60 DEL 2023\Nation ...,D:\MUSIQUE\PASAR A MAC\TOP 60 DEL 2023\Nation ...,file:///Users/legoteque/Music/Itunes%20Media/N...,/Volumes/PASAR A MAC/TOP 60 DEL 2023/Nation of...,True,True,\\169.254.170.1\Musica\Itunes Media\Nation of ...,5.0,red,red
1230,LOCAL,TOP 60 DEL 2023,True,60,1.0,Blur,Barbaric,4:09,MP3,2023-12-29 18:49:20,,D:\MUSIQUE\PASAR A MAC\TOP 60 DEL 2023\Blur - ...,D:\MUSIQUE\PASAR A MAC\TOP 60 DEL 2023\Blur - ...,file:///Users/legoteque/Music/Itunes%20Media/B...,/Volumes/PASAR A MAC/TOP 60 DEL 2023/Blur - Ba...,True,True,\\169.254.170.1\Musica\Itunes Media\Blur - Bar...,5.0,red,red
1231,LOCAL,TOP 60 DEL 2023,True,60,2.0,Spirits Of Saturn,Feelings Agency,6:10,MP3,2023-12-29 18:49:21,,D:\MUSIQUE\PASAR A MAC\TOP 60 DEL 2023\Spirits...,D:\MUSIQUE\PASAR A MAC\TOP 60 DEL 2023\Spirits...,file:///Users/legoteque/Music/Itunes%20Media/S...,/Volumes/PASAR A MAC/TOP 60 DEL 2023/Spirits O...,True,True,\\169.254.170.1\Musica\Itunes Media\Spirits Of...,5.0,red,red
1232,LOCAL,TOP 60 DEL 2023,True,60,3.0,Peace,Happy Cars,3:18,MP3,2023-12-29 18:49:29,,D:\MUSIQUE\PASAR A MAC\TOP 60 DEL 2023\Peace -...,D:\MUSIQUE\PASAR A MAC\TOP 60 DEL 2023\Peace -...,file:///Users/legoteque/Music/Itunes%20Media/P...,/Volumes/PASAR A MAC/TOP 60 DEL 2023/Peace - H...,True,True,\\169.254.170.1\Musica\Itunes Media\Peace - Ha...,5.0,red,red
1233,LOCAL,TOP 60 DEL 2023,True,60,4.0,English Teacher,Nearly Daffodils,3:45,MP3,2023-12-29 18:49:37,,D:\MUSIQUE\PASAR A MAC\TOP 60 DEL 2023\English...,D:\MUSIQUE\PASAR A MAC\TOP 60 DEL 2023\English...,file:///Users/legoteque/Music/Itunes%20Media/E...,/Volumes/PASAR A MAC/TOP 60 DEL 2023/English T...,True,True,\\169.254.170.1\Musica\Itunes Media\English Te...,5.0,red,red
1234,LOCAL,TOP 60 DEL 2023,True,60,5.0,Depeche Mode,Ghosts Again,3:59,MP3,2023-12-29 18:49:50,,D:\MUSIQUE\PASAR A MAC\TOP 60 DEL 2023\Depeche...,D:\MUSIQUE\PASAR A MAC\TOP 60 DEL 2023\Depeche...,file:///Users/legoteque/Music/Itunes%20Media/D...,/Volumes/PASAR A MAC/TOP 60 DEL 2023/Depeche M...,True,True,\\169.254.170.1\Musica\Itunes Media\Depeche Mo...,5.0,red,red
1235,LOCAL,TOP 60 DEL 2023,True,60,6.0,The Blaze,LONELY,4:25,MP3,2023-12-29 18:49:51,,D:\MUSIQUE\PASAR A MAC\TOP 60 DEL 2023\The Bla...,D:\MUSIQUE\PASAR A MAC\TOP 60 DEL 2023\The Bla...,file:///Users/legoteque/Music/Itunes%20Media/T...,/Volumes/PASAR A MAC/TOP 60 DEL 2023/The Blaze...,True,True,\\169.254.170.1\Musica\Itunes Media\The Blaze ...,5.0,red,red
1236,LOCAL,TOP 60 DEL 2023,True,60,7.0,Melenas,Bang,5:10,MP3,2023-12-29 18:50:02,,D:\MUSIQUE\PASAR A MAC\TOP 60 DEL 2023\Melenas...,D:\MUSIQUE\PASAR A MAC\TOP 60 DEL 2023\Melenas...,file:///Users/legoteque/Music/Itunes%20Media/M...,/Volumes/PASAR A MAC/TOP 60 DEL 2023/Melenas -...,True,True,\\169.254.170.1\Musica\Itunes Media\Melenas - ...,5.0,red,red
1237,LOCAL,TOP 60 DEL 2023,True,60,8.0,Water From Your Eyes,Barley,3:29,MP3,2023-12-29 18:50:04,,D:\MUSIQUE\PASAR A MAC\TOP 60 DEL 2023\Water F...,D:\MUSIQUE\PASAR A MAC\TOP 60 DEL 2023\Water F...,file:///Users/legoteque/Music/Itunes%20Media/W...,/Volumes/PASAR A MAC/TOP 60 DEL 2023/Water Fro...,True,True,\\169.254.170.1\Musica\Itunes Media\Water From...,5.0,red,red
1238,LOCAL,TOP 60 DEL 2023,True,60,9.0,DAIISTAR,Parallel,2:55,MP3,2023-12-29 18:50:17,,D:\MUSIQUE\PASAR A MAC\TOP 60 DEL 2023\DAIISTA...,D:\MUSIQUE\PASAR A MAC\TOP 60 DEL 2023\DAIISTA...,file:///Users/legoteque/Music/Itunes%20Media/D...,/Volumes/PASAR A MAC/TOP 60 DEL 2023/DAIISTAR ...,True,True,\\169.254.170.1\Musica\Itunes Media\DAIISTAR -...,5.0,red,red


In [4]:
#funció a la que passant un unique ens retorna els uniques sota un pattern
def uniques(column, pattern, genre=None):
    if genre == None:
        data = app.tracks_df[column].unique()
    else:
        data = app.tracks_df[app.tracks_df.Genre == genre]
        data = data[column].unique()
    matches = []
    for value in data:
        found = re.findall(pattern, value)
        matches = matches + found
        matches = set(matches)
        matches = list(matches)
    return matches

#hashtags unics
pattern = r"\#\S+"
data = uniques("Comments", pattern)

In [5]:
data

['#ta13',
 '#ta12',
 '#tL13',
 '#wut',
 '#nag',
 '#tL',
 '#rh',
 '#.',
 '#qc',
 '#bg',
 '#pC',
 '#wj',
 '#carnival',
 '#end',
 '#ska',
 '#teed',
 '#314',
 '#bp',
 '#electro',
 '#tL14',
 '#t',
 '#as',
 '#hi',
 '#re',
 '#7m',
 '#ta22',
 '#h',
 '#edit',
 '#wr',
 '#ge',
 '#al',
 '#intro',
 '#ta11',
 '#gl',
 '#fr',
 '#ta20',
 '#tL17',
 '#it',
 '#313',
 '#bb',
 '#wuj',
 '#dv',
 '#ef',
 '#ta19',
 '#ta17',
 '#ta14',
 '#ap',
 '#wu',
 '#halloween',
 '#sd',
 '#sw',
 '#st',
 '#yy',
 '#311',
 '#bi',
 '#az',
 '#mc',
 '#wt',
 '#dj',
 '#am',
 '#r',
 '#pd',
 '#cb',
 '#ca',
 '#tL20',
 '#tL18',
 '#tL15',
 '#wjr',
 '#df',
 '#ta21',
 '#mj',
 '#!!!',
 '#cat',
 '#cf',
 '#dma',
 '#ta18',
 '#fs',
 '#aj',
 '#tv',
 '#ta15',
 '#bk',
 '#rx',
 '#312',
 '#tH',
 '#ar',
 '#tW',
 '#tL16',
 '#ta',
 '#eC',
 '#tL22',
 '#ta16',
 '#dfa',
 '#tw',
 '#br',
 '#tL19']

In [3]:
app.lm.selected_list_df

,Folder,List,Smart,Length,Position,Artist,Name,Duration,Codec,Creation,Date Modified,Location,pc_loc,mac_loc,pc_mac_loc,linkable,linked,mac_pc_loc,level,color,title_song_color,folder_iid,list_iid,audio_iid
174,LOCAL,viktor,False,5,0,&ME;Rampa;Adam Port;Keinemusik;Ali Love,Confusion (Laolu Remix),6:44,MP3,2022-04-12 19:24:55,,D:\MUSIQUE\PASAR A MAC\Nueva carpeta\viktor\&M...,D:\MUSIQUE\PASAR A MAC\Nueva carpeta\viktor\&M...,file:///Users/legoteque/Music/Itunes%20Media/C...,/Volumes/PASAR A MAC/Nueva carpeta/viktor/&ME ...,True,False,\\169.254.170.1\Musica\Itunes Media\Cycle - Co...,1,sea green,blue,0,1,0
175,LOCAL,viktor,False,5,1,&ME;Rampa;Adam Port;Ali Love,Confusion,6:56,MP3,2022-04-12 19:24:50,,D:\MUSIQUE\PASAR A MAC\Nueva carpeta\viktor\&M...,D:\MUSIQUE\PASAR A MAC\Nueva carpeta\viktor\&M...,file:///Users/legoteque/Music/Itunes%20Media/C...,/Volumes/PASAR A MAC/Nueva carpeta/viktor/&ME ...,True,False,\\169.254.170.1\Musica\Itunes Media\Cycle - Co...,2,steel blue,blue,0,1,1
176,LOCAL,viktor,False,5,2,Alan Dixon,Rudy's Spector,6:14,MP3,2022-04-12 19:24:12,,D:\MUSIQUE\PASAR A MAC\Nueva carpeta\viktor\Al...,D:\MUSIQUE\PASAR A MAC\Nueva carpeta\viktor\Al...,file:///Users/legoteque/Music/Itunes%20Media/P...,/Volumes/PASAR A MAC/Nueva carpeta/viktor/Alan...,True,False,\\169.254.170.1\Musica\Itunes Media\Patrick Co...,2,steel blue,blue,0,1,2
177,LOCAL,viktor,False,5,3,Effin & Blindin,Compute (Raxon Remix),5:51,MP3,2022-04-12 19:23:27,,D:\MUSIQUE\PASAR A MAC\Nueva carpeta\viktor\Ef...,D:\MUSIQUE\PASAR A MAC\Nueva carpeta\viktor\Ef...,file:///Users/legoteque/Music/Itunes%20Media/S...,/Volumes/PASAR A MAC/Nueva carpeta/viktor/Effi...,True,False,\\169.254.170.1\Musica\Itunes Media\Soulwax - ...,1,sea green,blue,0,1,3
178,LOCAL,viktor,False,5,4,Effin & Blindin,Compute,5:29,MP3,2022-04-12 19:23:24,,D:\MUSIQUE\PASAR A MAC\Nueva carpeta\viktor\Ef...,D:\MUSIQUE\PASAR A MAC\Nueva carpeta\viktor\Ef...,file:///Users/legoteque/Music/Itunes%20Media/S...,/Volumes/PASAR A MAC/Nueva carpeta/viktor/Effi...,True,False,\\169.254.170.1\Musica\Itunes Media\Soulwax - ...,2,steel blue,blue,0,1,4


In [6]:
app.ct._track_dic

{'Folder': 'LOCAL',
 'List': 'viktor',
 'Smart': False,
 'Length': 5,
 'Position': 3,
 'Artist': 'Effin & Blindin',
 'Name': 'Compute (Raxon Remix)',
 'Duration': '5:51',
 'Codec': 'MP3',
 'Creation': '2022-04-12 19:23:27',
 'Date Modified': '',
 'Location': 'D:\\MUSIQUE\\PASAR A MAC\\Nueva carpeta\\viktor\\Effin & Blindin - Compute (Raxon Remix).mp3',
 'pc_loc': 'D:\\MUSIQUE\\PASAR A MAC\\Nueva carpeta\\viktor\\Effin & Blindin - Compute (Raxon Remix).mp3',
 'mac_loc': 'file:///Users/legoteque/Music/Itunes%20Media/Soulwax%20-%20Compute%20%20(Nite%20Versions).mp3',
 'pc_mac_loc': '/Volumes/PASAR A MAC/Nueva carpeta/viktor/Effin & Blindin - Compute (Raxon Remix).mp3',
 'linkable': True,
 'linked': False,
 'mac_pc_loc': '\\\\169.254.170.1\\Musica\\Itunes Media\\Soulwax - Compute  (Nite Versions).mp3',
 'level': 1,
 'color': 'sea green',
 'title_song_color': 'blue',
 'folder_iid': '0',
 'list_iid': '1',
 'audio_iid': '3'}

In [4]:
app.playlists_df[app.playlists_df.Folder == "LOCAL"]

,Folder,List,Smart,Length,Position,Artist,Name,Duration,Codec,Creation,Date Modified,Location,pc_loc,mac_loc,pc_mac_loc,linkable,linked,mac_pc_loc,level,color,title_song_color,folder_iid,list_iid,audio_iid
0,LOCAL,Nueva carpeta2,False,174,0,Jade,Pulp Friction,4:55,MP3,2022-11-14 22:30:26,,D:\MUSIQUE\PASAR A MAC\Nueva carpeta\Nueva car...,D:\MUSIQUE\PASAR A MAC\Nueva carpeta\Nueva car...,,/Volumes/PASAR A MAC/Nueva carpeta/Nueva carpe...,False,False,,0,darkgreen,blue,,,
1,LOCAL,Nueva carpeta2,False,174,1,Pitbull;Christina Aguilera,Feel This Moment (feat. Christina Aguilera),3:50,MP3,2022-11-14 22:30:23,,D:\MUSIQUE\PASAR A MAC\Nueva carpeta\Nueva car...,D:\MUSIQUE\PASAR A MAC\Nueva carpeta\Nueva car...,file:///Users/legoteque/Music/Itunes%20Media/P...,/Volumes/PASAR A MAC/Nueva carpeta/Nueva carpe...,True,True,\\169.254.170.1\Musica\Itunes Media\Pitbull - ...,5,red,red,,,
2,LOCAL,Nueva carpeta2,False,174,2,Quinten Circle,Listen To Me Now,2:33,MP3,2022-11-14 22:30:22,,D:\MUSIQUE\PASAR A MAC\Nueva carpeta\Nueva car...,D:\MUSIQUE\PASAR A MAC\Nueva carpeta\Nueva car...,file:///Users/legoteque/Music/Itunes%20Media/F...,/Volumes/PASAR A MAC/Nueva carpeta/Nueva carpe...,True,False,\\169.254.170.1\Musica\Itunes Media\Fufanu - L...,1,sea green,blue,,,
3,LOCAL,Nueva carpeta2,False,174,3,The Blaze,DREAMER,5:39,MP3,2022-11-12 21:50:57,,D:\MUSIQUE\PASAR A MAC\Nueva carpeta\Nueva car...,D:\MUSIQUE\PASAR A MAC\Nueva carpeta\Nueva car...,file:///Users/legoteque/Music/Itunes%20Media/T...,/Volumes/PASAR A MAC/Nueva carpeta/Nueva carpe...,True,False,\\169.254.170.1\Musica\Itunes Media\The Blaze ...,2,steel blue,blue,,,
4,LOCAL,Nueva carpeta2,False,174,4,Rachel Santos,Sará Perché Ti Amo,3:06,MP3,2022-11-12 17:02:04,,D:\MUSIQUE\PASAR A MAC\Nueva carpeta\Nueva car...,D:\MUSIQUE\PASAR A MAC\Nueva carpeta\Nueva car...,file:///Users/legoteque/Music/Itunes%20Media/D...,/Volumes/PASAR A MAC/Nueva carpeta/Nueva carpe...,True,False,\\169.254.170.1\Musica\Itunes Media\Dj Matrix ...,2,steel blue,blue,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174,LOCAL,viktor,False,5,0,&ME;Rampa;Adam Port;Keinemusik;Ali Love,Confusion (Laolu Remix),6:44,MP3,2022-04-12 19:24:55,,D:\MUSIQUE\PASAR A MAC\Nueva carpeta\viktor\&M...,D:\MUSIQUE\PASAR A MAC\Nueva carpeta\viktor\&M...,file:///Users/legoteque/Music/Itunes%20Media/C...,/Volumes/PASAR A MAC/Nueva carpeta/viktor/&ME ...,True,False,\\169.254.170.1\Musica\Itunes Media\Cycle - Co...,1,sea green,blue,0,0,0
175,LOCAL,viktor,False,5,1,&ME;Rampa;Adam Port;Ali Love,Confusion,6:56,MP3,2022-04-12 19:24:50,,D:\MUSIQUE\PASAR A MAC\Nueva carpeta\viktor\&M...,D:\MUSIQUE\PASAR A MAC\Nueva carpeta\viktor\&M...,file:///Users/legoteque/Music/Itunes%20Media/C...,/Volumes/PASAR A MAC/Nueva carpeta/viktor/&ME ...,True,False,\\169.254.170.1\Musica\Itunes Media\Cycle - Co...,2,steel blue,blue,0,0,1
176,LOCAL,viktor,False,5,2,Alan Dixon,Rudy's Spector,6:14,MP3,2022-04-12 19:24:12,,D:\MUSIQUE\PASAR A MAC\Nueva carpeta\viktor\Al...,D:\MUSIQUE\PASAR A MAC\Nueva carpeta\viktor\Al...,file:///Users/legoteque/Music/Itunes%20Media/P...,/Volumes/PASAR A MAC/Nueva carpeta/viktor/Alan...,True,False,\\169.254.170.1\Musica\Itunes Media\Patrick Co...,2,steel blue,blue,0,0,2
177,LOCAL,viktor,False,5,3,Effin & Blindin,Compute (Raxon Remix),5:51,MP3,2022-04-12 19:23:27,,D:\MUSIQUE\PASAR A MAC\Nueva carpeta\viktor\Ef...,D:\MUSIQUE\PASAR A MAC\Nueva carpeta\viktor\Ef...,file:///Users/legoteque/Music/Itunes%20Media/S...,/Volumes/PASAR A MAC/Nueva carpeta/viktor/Effi...,True,False,\\169.254.170.1\Musica\Itunes Media\Soulwax - ...,1,sea green,blue,0,0,4


In [10]:
mutagen.File(app.ct.mac_pc_loc).info.length

365.81333333333333

In [9]:
app.ct.mac_pc_loc

'\\\\169.254.170.1\\Musica\\Itunes Media\\Soulwax - Compute  (Nite Versions).mp3'

In [6]:
app.lm.current_list_df

,Folder,List,Smart,Length,Position,Artist,Name,Duration,Codec,Creation,Date Modified,Location,pc_loc,mac_loc,pc_mac_loc,linkable,link,mac_pc_loc,level,color,title_song_color,folder_iid,list_iid,audio_iid
0,LOCAL,viktor,False,6,0,&ME;Rampa;Adam Port;Keinemusik;Ali Love,Confusion (Laolu Remix),6:44,MP3,2022-04-12 19:24:55,,D:\MUSIQUE\PASAR A MAC\Nueva carpeta\viktor\&M...,D:\MUSIQUE\PASAR A MAC\Nueva carpeta\viktor\&M...,file:///Users/legoteque/Music/Itunes%20Media/C...,/Volumes/PASAR A MAC/Nueva carpeta/viktor/&ME ...,True,False,\\169.254.170.1\Musica\Itunes Media\Cycle - Co...,1,sea green,blue,0,0,0
1,LOCAL,viktor,False,6,1,&ME;Rampa;Adam Port;Ali Love,Confusion,6:56,MP3,2022-04-12 19:24:50,,D:\MUSIQUE\PASAR A MAC\Nueva carpeta\viktor\&M...,D:\MUSIQUE\PASAR A MAC\Nueva carpeta\viktor\&M...,file:///Users/legoteque/Music/Itunes%20Media/C...,/Volumes/PASAR A MAC/Nueva carpeta/viktor/&ME ...,True,False,\\169.254.170.1\Musica\Itunes Media\Cycle - Co...,2,steel blue,blue,0,0,1
2,LOCAL,viktor,False,6,2,Alan Dixon,Rudy's Spector,6:14,MP3,2022-04-12 19:24:12,,D:\MUSIQUE\PASAR A MAC\Nueva carpeta\viktor\Al...,D:\MUSIQUE\PASAR A MAC\Nueva carpeta\viktor\Al...,file:///Users/legoteque/Music/Itunes%20Media/P...,/Volumes/PASAR A MAC/Nueva carpeta/viktor/Alan...,True,False,\\169.254.170.1\Musica\Itunes Media\Patrick Co...,2,steel blue,blue,0,0,2
5,LOCAL,viktor,False,6,5,RÜFÜS DU SOL,Underwater (Adam Port Remix),6:39,MP3,2022-04-12 19:23:51,,D:\MUSIQUE\PASAR A MAC\Nueva carpeta\viktor\RÜ...,D:\MUSIQUE\PASAR A MAC\Nueva carpeta\viktor\RÜ...,file:///Users/legoteque/Music/Itunes%20Media/R...,/Volumes/PASAR A MAC/Nueva carpeta/viktor/RÜFÜ...,True,True,\\169.254.170.1\Musica\Itunes Media\Rüfüs du...,11,red,red,0,0,3
3,LOCAL,viktor,False,6,3,Effin & Blindin,Compute (Raxon Remix),5:51,MP3,2022-04-12 19:23:27,,D:\MUSIQUE\PASAR A MAC\Nueva carpeta\viktor\Ef...,D:\MUSIQUE\PASAR A MAC\Nueva carpeta\viktor\Ef...,file:///Users/legoteque/Music/Itunes%20Media/S...,/Volumes/PASAR A MAC/Nueva carpeta/viktor/Effi...,True,False,\\169.254.170.1\Musica\Itunes Media\Soulwax - ...,1,sea green,blue,0,0,4
4,LOCAL,viktor,False,6,4,Effin & Blindin,Compute,5:29,MP3,2022-04-12 19:23:24,,D:\MUSIQUE\PASAR A MAC\Nueva carpeta\viktor\Ef...,D:\MUSIQUE\PASAR A MAC\Nueva carpeta\viktor\Ef...,file:///Users/legoteque/Music/Itunes%20Media/S...,/Volumes/PASAR A MAC/Nueva carpeta/viktor/Effi...,True,False,\\169.254.170.1\Musica\Itunes Media\Soulwax - ...,2,steel blue,blue,0,0,5


In [ ]:
with Capturing() as output:
    print('hello world')

print('displays on screen')

with Capturing(output) as output:  # note the constructor argument
    print('hello world2')

print('done')
print('output:', output)